# RateCurrent_Automation
**Rates packages for small parcel customers under the actual services (UPS, DHL, USPS, and UPS Intl).**

**Structure**:  
Section 1. Read in and Re-format Freight Rates  
Section 2. Read in and Re-format all other 4 input tables  
Section 3. Re-format and Rate PLD
           
           a. Load PLD from snowflake query
           b. Patch PLD
           c. Flag PLD (Flag instead of delete data)
           d. Add columns to PLD
           e. Add Freight Rates (Published Freight "UPS only" & Barrett Freight "except UPS") to PLD
           f. Add Customer Freight to PLD (and Barrett Freight "UPS only" to PLD)
           g. Add UPS Published & Barrett & Customer Resi, Das, and Ahs fee
           h. Add dhl & usps accessorials
           i. Get FSC (only for UPS and DHL)
           j. Get Total
           k. Handle Special Cases and Flag Rate
           l. Calculate the savings and Savings % and Format


**Input tables**:  
Doc. 1. FreightCharge_xxxx: freight rate cards (insert updated version once a year)  
Doc. 2. ClientCharge_xxxx: client rate and parameters (insert newer version once a quarter at Max.)  
Doc. 3. FuelSurcharge_weekly: fuel surcharge rate (update once a week)  
Doc. 4. ZipToZone: get zone from destination zip based on different facilities  
Doc. 5. ZipToDas_xxxx: a table to assign the type of DAS Category (insert newer version once a year)  
Doc. 6. PLD: snowflake package level detail as an input table



**Input Data Cleaning & Formatting**

| Column Input | Column Type | Function | Null Case | Error Case   |
|:--------|:---------|:-------------|:------------|:------------|
| TrackingNumber     | any (int64, object, etc)  | filter out duplications         | fill out number from 1,2,...         | no concern   |
| Facility & ZipCode     | object                                                                                                              | get zone   | no concern if Zone exists         | no concern if Zone exists   |
| ShipDate     | datetime                                                                                                             | get fsc%  | use yesterday's Date         | use yesterday's Date   |
| Service     | object                                                                                                                | get target service for rate         | cannot rate         | cannot rate   |
| ZipCode     | object | get das | cannot rate | cannot rate |
| Zone     | object | get Freight | cannot rate if “Zip Code” and “Facility” do not exist | cannot rate if “Zip Code” and “Facility” do not exist |
| Dimensions    | object | get billed weight | cannot rate | cannot rate | 
| ActualWeight    | float | get billed weight | cannot rate | cannot rate 0 lb or over 150 lbs|
| ResidentialFlag    | bool| get target service for rate | no concern | no concern |


**FreightCharge note**

| Carrier | Service | Residential | WeightType | RateType   |
|:--------|:---------|:-------------|:------------|:------------|
| UPS     | REDE (UPS NDA Early) / RED (UPS NDA) / REDS (UPS NDA Saver) / 2DAM (UPS 2DA A.M.) / BLUE (UPS 2DA) / ORNG (UPS 3DA) / SRPT (UPS Surepost) / CNST (UPS Standard to Canada) / WWE (World Wide Expedited) / WWXS (World Wide Express Saver) / WWX (World Wide Express)  | both         | lb         | Published   |
| UPS     | GRND (UPS Ground Commercial)                                                                                                              | commercial   | lb         | Published   |
| UPS     | GRES (UPS Ground Residential)                                                                                                             | residential  | lb         | Published   |
| UPS     | SRPT<1 (UPS Surepost 1#>)                                                                                                                 | both         | oz         | Published   |
| DHL     | DHLG (DHL SmartMail Parcel Plus Ground 1-25)/DHLE (DHL SmartMail Parcel Plus Expedited 1-25) | both | lb | Barrett |
| DHL     | DHLG<1 (DHL SmartMail Parcel Ground < 1lb)/DHLE<1 (DHL SmartMail Parcel Expedited  < 1lb)/DHLEM (DHL SmartMail Parcel Expedited Max) | both | oz | Barrett |
| USPS    | USPSAP_CPP (USPS Auctane PM) | both | lb* | Barrett* | 
| USPS    | USPSFC_CPP (USPS First Class) | both | oz | Barrett* |
| USPS    | USPSPS_CPP (USPS Parcel Select)| both | lb | Barrett* |
| USPS    | USPSAP_AUCT (USPS Auctane PM) | both | lb* | Barrett | 
| USPS    | USPSFC_AUCT (USPS First Class) | both | oz | Barrett |
| USPS    | USPSPS_AUCT (USPS Parcel Select)| both | lb | Barrett |
| MI      | MIPLE (MI Parcel Select Lightweight Expedited)|  both | oz | Barrett |
| MI      | MIPH (MI Parcel Select Heavyweight)|  both | lb | Barrett |

*For convenience of exchange, flag as Barrett Rate. Although AP has one "<1lb" weight but still flagged as "lb". 

**Client rate calculation based on "Barrett Dim" Pub rate and Barrett rate**

| Carrier | Service | Client Rate Calculation   |
|:--------|:---------|:-------------|
| UPS     | REDE / RED / REDS / 2DAM / BLUE / ORNG / CNST / WWE / WWXS / WWX |  max(Pub Rate *(1 - Round(Discount%,2)), Min) ** |
| UPS     | GRND / SRPT / GRES  |  max(Pub Rate *(1 - Round(Discount%,2)), Min) **Consider weight break in Lb |
| UPS     | SRPT<1  |  max(Pub Rate *(1 - Round(Discount%,2)), Min) **Consider weight break in Oz |
| DHL/MI | All | Barrett Rate / (1-Margin%) | 
| USPS | All | CPP or Barrett Rate / (1-Margin%) | 

*The output breakdown rates are rounded to 2 decimal places. The total rate is a sum up of the breakdowns and then rounded again. And the Savings and Margin are rounded again upon the calculation of the other columns.  
*4172 applies to 417801 and 4178 (417801 and 4178 do not exist in data)  
*The "version switch + number" located in this code flags the place to edit to switch to a different version. 

*********Parameter Set Up*********

In [38]:
import pandas as pd
import numpy as np
import math
import re
from datetime import timedelta, date
from datetime import datetime, timedelta

#Dim factor
dim_factor_ups_pub = 139
dim_factor_ups = 280
dim_factor_ups_srpt = 139
dim_factor_ups_intl = 139
dim_factor_dhl = 166
dim_factor_usps = 166
dim_factor_mi = 166

#Dim Allowance
ups_allowance = 3456
dhl_allowance = 1728
usps_allowance = 1728
mi_allowance = 1728

#ups parameters
ups_fsc_disc = 0.8
intl_ground_res = 5.5 #5.85 in 2024 (version switch 1)
intl_air_res = 5.85 #6.20 in 2024 (version switch 2)
intl_ahs_weight_limit = 70 #55 in 2024 (version switch 3)
intl_ahs = 24 #31 in 2024 (version switch 4)

#dhl parameters
dhl_cubic_inch_threshold_1 = 1728
dhl_cubic_inch_threshold_2 = 3456
dhl_weight_threshold = 1
dhl_g_and_l_threshold_1 = 50
dhl_g_and_l_threshold_2 = 84
dhl_max_l_threshold_1 = 22
dhl_max_l_threshold_2 = 27
dhl_max_l_threshold_3 = 30

dhl_nqd_lengthfee_1 = 4.5
dhl_nqd_lengthfee_2 = 15.5
dhl_nqd_volumefee = 15.5

# update to 2024 when needed
dates = ['2023-01-01', '2023-09-30', '2023-10-01', '2023-12-31'] #(version switch 5)
nqd_dhl_date_1, nqd_dhl_date_2, nqd_dhl_date_3, nqd_dhl_date_4 = [pd.to_datetime(date) for date in dates]

#usps parameters
usps_l_threshold_1 = 22
usps_l_threshold_2 = 30
usps_cubic_inch_threshold = 3456

usps_nonestandard_lengthfee_1 = 4 #2024: 4 (version switch 6)
usps_nonestandard_lengthfee_2 = 7 #2024: 18 (version switch 7)
usps_nonestandard_volumefee = 15 #2024: 30 (version switch 8)


# Barrett min international charge 2023
mapping_min_br_intl_2023 = {# Canada Standard
                '51': 15.61, '52': 15.61, '53': 15.61, '54': 15.61, '55': 15.61, '56': 15.61,
    # World Wide Expedited
                '71': 29.98, '72': 31.76, '74': 26.58, '601/631': 34.59, '602/632': 28.37, '603/633': 37.54, '604/634': 40.24, '605/635': 35.10,
'606/636': 47.57, '607/637': 39.33, '608/638': 36.60, '609/639': 33.30, '611/641': 36.95, '612/642': 36.79, '613/643': 32.19,
'620': 27.66, '621': 30.79, # World Wide Express Saver, 
                '481': 31.569, '482': 34.899, '484': 32.157, '401': 38.868, '402': 34.44, '403': 44.517, '404': 42.48, '405': 41.076,
'406': 51.042, '407': 61.182, '408': 63.648, '409': 36.132, '411': 43.464, '412': 40.509, '413': 35.625, '420': 32.718,
'421': 33.849, # World Wide Express
                '81': 32.952, '82': 36.297, '84': 32.769, '901': 40.035, '902': 34.956, '903': 46.5, '904': 43.035, '905': 41.721, 
'906': 51.759, '907': 63.147, '908': 64.563, '909': 36.603, '911': 44.097, '912': 41.877, '913': 37.083, '920': 33.21,
'921': 34.911}


# Barrett min international charge 2024
mapping_min_br_intl_2024 = {# Canada Standard
                '51': 16.52, '52': 16.52, '53': 16.52, '54': 16.52, '55': 16.52, '56': 16.52,
    # World Wide Expedited
                '71': 31.48, '72': 33.50, '74': 27.91, '601/631': 36.32, '602/632': 28.66, '603/633': 37.54, '604/634': 41.85, '605/635': 37.21,
'606/636': 51.38, '607/637': 39.33, '608/638': 36.60, '609/639': 35.30, '611/641': 38.80, '612/642': 39.36, '613/643': 34.12,
'620': 28.44, '621': 32.09, # World Wide Express Saver, 
                '481': 33.62, '482': 36.99, '484': 34.09, '401': 41.59, '402': 37.20, '403': 47.63, '404': 44.60, '405': 43.95,
'406': 53.60, '407': 65.47, '408': 69.38, '409': 38.66, '411': 47.38, '412': 42.54, '413': 37.05, '420': 34.00,
'421': 35.17, # World Wide Express
                '81': 34.97, '82': 38.84, '84': 35.28, '901': 42.63, '902': 39.06, '903': 49.54, '904': 46.39, '905': 46.15, 
'906': 55.74, '907': 68.74, '908': 72.85, '909': 40.60, '911': 50.22, '912': 45.09, '913': 38.72, '920': 35.66,
'921': 36.90}

# update to 2023 or 2024 when needed
mapping_min_br_intl = mapping_min_br_intl_2023 #(version switch 9)

**Section 1. Read in and Re-format Freight Rates**

In [39]:
# *************** Read Doc. 1 *************************
xls = pd.ExcelFile('FreightCharge_2023.xlsx') # (version switch 10)

# Group all rate cards based on their WeightType & RateType & handle DHL independently
# Lb and Published Rate
tabs_1 = ['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG', 'GRND', 'GRES', 'SRPT', 'CNST', 'WWE', 'WWXS', 'WWX']
# Lb and Barrett Rate
tabs_2 = ['USPSPS_AUCT', 'USPSAP_AUCT', 'USPSPS_CPP', 'USPSAP_CPP', 'MIPH']
# Oz and Published Rate
tabs_3 = ['SRPT<1'] 
# Oz and Barrett Rate
tabs_4 = ['USPSFC_AUCT', 'USPSFC_CPP', 'MIPLE']
# Lb and Barrett Rate (DHL)
dhl_ids_1 = ['5114358', '5122444', '5122723', '5122739', '5122855', '5122890', '5122893', '5123280', '5123282', '5123283'#, '5123556' # (version switch 11) no '5123556' in 2023 but in 2024
            ]
dhl_types_1 = ['DHLG', 'DHLE']
dhl_tabs_1 = [f"{type}_{id}" for id in dhl_ids_1 for type in dhl_types_1]
# Oz and Barrett Rate (DHL)
dhl_ids_2 = ['5114358', '5122444', '5122723', '5122739', '5122855', '5122890', '5122893', '5123280', '5123282', '5123283'#, '5123556' # (version switch 12) no '5123556' in 2023 but in 2024
            ]
dhl_types_2 = ['DHLG<1', 'DHLE<1', 'DHLEM']
dhl_tabs_2 = [f"{type}_{id}" for id in dhl_ids_2 for type in dhl_types_2 if not (type == 'DHLEM' and id == '5122444')  # (version switch 13) DHLEM & '5122444' exists in 2024 but not in 2023
             ]

# Combine all the groups into a single list
all_tabs = tabs_1 + tabs_2 + tabs_3 + tabs_4 + dhl_tabs_1 + dhl_tabs_2

# Regroup/re-categorize the tabs into 4 groups
lb_br_tabs = tabs_2 + dhl_tabs_1    
lb_pr_tabs = tabs_1
oz_br_tabs = tabs_4 + dhl_tabs_2    
oz_pr_tabs = tabs_3

# Load data using a dictionary comprehension. dfs['REDE']
dfs = {tab: pd.read_excel(xls, tab) for tab in all_tabs}

# Process each dataframe based on tab name
for tab, df in dfs.items():
    # Determine weight column name
    if tab in oz_br_tabs or tab in oz_pr_tabs:
        weight_col = 'WeightOz'
    elif tab in lb_br_tabs or tab in lb_pr_tabs:
        weight_col = 'WeightLb'
    else:
        raise ValueError(f"Tab {tab} is not categorized properly by weight.")
    
    # Determine rate column name
    if tab in oz_br_tabs or tab in lb_br_tabs:
        rate_col_name = 'BarrettRate'
    elif tab in oz_pr_tabs or tab in lb_pr_tabs:
        rate_col_name = 'PublishedRate'
    else:
        raise ValueError(f"Tab {tab} is not categorized properly by rate.")

    # Apply unstack operation
    dfs[tab] = (df.set_index('Zones')
                .unstack()
                .rename_axis(('Zone', weight_col))
                .reset_index(name=rate_col_name))

# Concatenate all transformed dataframes into one, with an additional column indicating the tab name
FreightCharge = pd.concat(dfs.values(), keys=dfs.keys(), ignore_index=False)
FreightCharge.reset_index(level=0, inplace=True)
FreightCharge.rename(columns={'level_0': 'ServiceCode'}, inplace=True)

# Fill missing values in 'Weight_Lb' using 'Weight_Oz' / 16
FreightCharge['WeightLb'] = np.where(
    FreightCharge['WeightLb'].isna(), 
    FreightCharge['WeightOz'] / 16,     
    FreightCharge['WeightLb']           
)

# Fill missing values in 'Weight_Oz' using 'Weight_Lb' * 16
FreightCharge['WeightOz'] = np.where(
    FreightCharge['WeightOz'].isna(),  
    FreightCharge['WeightLb'] * 16,    
    FreightCharge['WeightOz']          
)


# Generate Service detailed names based on ServiceCode section
subset_numbers = ['5114358', '5122723', '5122739', '5122855', '5122890', '5122893', '5123280', '5123282', '5123283'# , '5123556' # (version switch 14) '5123556' exists in 2024 but not in 2023
                 ]

def generate_service_details(ServiceCode):
    number = ServiceCode.split('_')[-1]
    if number not in subset_numbers:
        return None

    if "DHLG_" in ServiceCode:
        return f'DHL SmartMail Parcel Plus Ground 1-25 {number}'
    elif "DHLG<1_" in ServiceCode:
        return f'DHL SmartMail Parcel Ground < 1lb {number}'
    elif "DHLE_" in ServiceCode:
        return f'DHL SmartMail Parcel Plus Expedited 1-25 {number}'
    elif "DHLE<1_" in ServiceCode:
        return f'DHL SmartMail Parcel Expedited < 1lb {number}'
    elif "DHLEM_" in ServiceCode:
        return f'DHL SmartMail Parcel Expedited Max {number}'
    else:
        return None

FreightCharge['Service'] = FreightCharge['ServiceCode'].apply(generate_service_details)

tab_to_description = {
    'REDE': 'UPS NDA Early',
    'RED': 'UPS NDA',
    'REDS': 'UPS NDA Saver',
    '2DAM': 'UPS 2DA A.M.',
    'BLUE': 'UPS 2DA',
    'ORNG': 'UPS 3DA',
    'GRND': 'UPS Ground Commercial',
    'GRES': 'UPS Ground Residential',
    'SRPT<1': 'UPS Surepost 1#>',
    'SRPT': 'UPS Surepost',
    'CNST': 'UPS Standard to Canada',
    'WWE': 'UPS Worldwide Express',
    'WWXS': 'UPS Worldwide Express Saver',
    'WWX': 'UPS Worldwide Expedited',
    'USPSFC_CPP': 'USPS First Class CPP',
    'USPSPS_CPP': 'USPS Parcel Select CPP',
    'USPSAP_CPP': 'USPS Priority Mail CPP',
    'USPSFC_AUCT': 'USPS First Class Auctane',
    'USPSPS_AUCT': 'USPS Parcel Select Auctane',
    'USPSAP_AUCT': 'USPS Priority Mail Auctane',
    'MIPLE': 'MI Parcel Select Lightweight Expedited',
    'MIPH': 'MI Parcel Select Heavyweight',
    'DHLG_5122444': 'DHL SmartMail Parcel Plus Ground 1-25 5122444',
    'DHLG<1_5122444': 'DHL SmartMail Parcel Ground < 1lb 5122444',
    'DHLE_5122444': 'DHL SmartMail Parcel Plus Expedited 1-25 5122444',
    'DHLE<1_5122444': 'DHL SmartMail Parcel Expedited < 1lb 5122444'
    #,'DHLEM_5122444': 'DHL SmartMail Parcel Expedited Max 5122444' # (version switch 15) 'DHLEM_5122444' exists in 2024 but not in 2023
    
}


# Function to update the rest of the Services
def update_service_details(row):
    # If the Service is empty or None
    if pd.isnull(row['Service']) or row['Service'] == "":
        return tab_to_description.get(row['ServiceCode'], row['Service'])
    else:
        # Return existing value
        return row['Service']

FreightCharge['Service'] = FreightCharge.apply(update_service_details, axis=1)

# Add the version column. Needs to update once a year.  (version switch 16)
FreightCharge['Version'] ='2023'
FreightCharge['StartDate'] ='2023/01/01'
FreightCharge['EndDate'] ='2023/12/31'

# Define the replacement mapping
replacement_mapping = {
    '102': '2', '103': '3', '104': '4', '105': '5', '106': '6', '107': '7', '108': '8', 
    '202': '2', '203': '3', '204': '4', '205': '5', '206': '6', '207': '7', '208': '8', 
    '302': '2', '303': '3', '304': '4', '305': '5', '306': '6', '307': '7', '308': '8',
    '44': '9', '45': '10', '46': '11', '124': '9', '125': '10', '126': '11', '224': '9', '225': '10', '226': '11',
    '03': '3', '04': '4', '05': '5', '06': '6', '07': '7', '08': '8', '09': '9'
}

def get_portion(value, replacement_mapping):
    # Convert value to string to ensure compatibility with re.match
    value_str = str(value)
    
    if re.match(r'\d+/\d+', value_str):
        return value_str  # Return the value as-is

    if re.match(r'[49]\d\d$', value_str):
        return value_str  # Keep it as it is
    
    # Original processing logic
    if 'Zone' in value_str:
        modified_value = value_str[5:]
    elif 'US' in value_str:
        modified_value = value_str[3:] 
        # Check if the value is in the format "xxx/xxx"
    else:
        modified_value = value_str
        
    # Apply replacements from the mapping
    for key, replacement in replacement_mapping.items():
        modified_value = modified_value.replace(key, replacement)

    return modified_value


# Apply the modified get_portion function to each value in the Zone column
FreightCharge['ZONE_new'] = FreightCharge['Zone'].apply(lambda x: get_portion(x, replacement_mapping))


FreightCharge['Zone'] = FreightCharge['ZONE_new']
FreightCharge = FreightCharge.drop('ZONE_new', axis=1)


#adjust the index and finish re-formatting this table.
FreightCharge.reset_index(drop=True, inplace=True)
FreightCharge = FreightCharge[['ServiceCode', 'Service', 'Zone', 'WeightOz', 'WeightLb', 'PublishedRate', 'BarrettRate', 'Version', 'StartDate', 'EndDate']]
#FreightCharge['Zone'] = FreightCharge['Zone'].astype('int64')
FreightCharge['Version'] = FreightCharge['Version'].astype('int64')

#check the table type and content
#FreightCharge.info() 
#FreightCharge.head()


# Keep rows where 'ServiceCode' is not in ['CPP']. CPP rate is less likely to be used than AUCT rate. 
FreightCharge = FreightCharge[~FreightCharge['ServiceCode'].isin(['USPSFC_CPP', 'USPSPS_CPP', 'USPSAP_CPP'])]
         
# Replacing values.
replacement_dict = {
    'USPSFC_AUCT': 'USPSFC',
    'USPSPS_AUCT': 'USPSPS',
    'USPSAP_AUCT': 'USPSAP'
}

# Replace the values in 'ServiceCode' column
FreightCharge['ServiceCode'] = FreightCharge['ServiceCode'].replace(replacement_dict)


In [40]:
#Run 2023 rates and save to FreightCharge_2023
#FreightCharge_2023 = FreightCharge

In [41]:
#Run 2024 rates and save to FreightCharge_2024
#FreightCharge_2024 = FreightCharge
#Combine 2023 and 2024. 
#FreightChargeMaster = pd.concat([FreightCharge_2023, FreightCharge_2024])
#Select the version to use in this model
#FreightCharge = FreightChargeMaster[FreightChargeMaster['Version'] == '2023'] # (version switch 17)

**Section 2. Read in and Re-format all other 4 input tables**

In [42]:
# ********** Read Doc. 2 **********
ClientCharge_2023 = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\ClientCharge_2023.xlsx')
ClientCharge_2024 = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\ClientCharge_2024.xlsx')
ClientChargeMaster = pd.concat([ClientCharge_2023, ClientCharge_2024])
#Select the version to use in this model
ClientCharge = ClientChargeMaster[ClientChargeMaster['StartDate'] == '2023-01-01'] # (version switch 18)

In [43]:
# ********** Read Doc. 3 **********
xls = pd.ExcelFile('FuelSurcharge_weekly.xlsx')

# Re-format the original fsc input doc by adding the range of period based on Ship Date
def prepare_FSC_data(df, type="UPS"):
    # Convert Ship Date to datetime once
    df['Ship Date'] = pd.to_datetime(df['Ship Date'])
    
    df['ShipDateStart'] = df['Ship Date']
    
    if type == "UPS":
        df['ShipDateEnd'] = df['ShipDateStart'] + pd.Timedelta(days=6)
    elif type == "DHL":
        df['ShipDateEnd'] = df['ShipDateStart'].shift(-1) - timedelta(days=1)
        df.loc[df['ShipDateStart'] == max(df['ShipDateStart']), ['ShipDateEnd']] = date.today()

    # Drop Ship Date column
    df.drop('Ship Date', axis=1, inplace=True)
    
    return df


UPS_FSC = prepare_FSC_data(pd.read_excel(xls, 'UPS_FSC'), type="UPS")
DHL_FSC = prepare_FSC_data(pd.read_excel(xls, 'DHL_FSC').sort_values(by='Ship Date', ascending=True), type="DHL")


In [44]:
# PLD already has a Zone column, but we still need to fill out the empty Zone values in PLD. 
# ********** Read Doc. 4 **********
xls = pd.ExcelFile('ZipToZone.xlsx')

# List of sheet names. Using a dictionary comprehension to load sheets into a dictionary of dataframes and then concatenate togeth
sheets = [
    '02038_Outbound', '08873_Outbound', '21226_Outbound', '38141_Outbound', '38654_Outbound', 
    '75041_Outbound', '90640_Outbound', '02324_Outbound', '06096_Outbound', '18105_Outbound', 
    '21240_Outbound', '43004_Outbound'
]

dfs = {sheet: pd.read_excel(xls, sheet) for sheet in sheets}

ZipToZone = pd.concat([dfs['02038_Outbound'], dfs['08873_Outbound'], dfs['21226_Outbound'], dfs['38141_Outbound'], dfs['38654_Outbound'], 
                       dfs['75041_Outbound'], dfs['90640_Outbound'], dfs['02324_Outbound'], dfs['06096_Outbound'], dfs['18105_Outbound'], 
                       dfs['21240_Outbound'], dfs['43004_Outbound']])


In [45]:
# ********** Read Doc. 5 ********** # get 2023 or 2024 as needed. 
ZipToDas_2023 = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\ZipToDas_2023.xlsx')
ZipToDas_2024 = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\ZipToDas_2024.xlsx')
ZipToDas_2023['Version'] = '2023'
ZipToDas_2024['Version'] = '2024'
ZipToDasMaster = pd.concat([ZipToDas_2023, ZipToDas_2024])
#Select the version to use in this model
ZipToDas = ZipToDasMaster[ZipToDasMaster['Version'] == '2023'] # (version switch 19final)

**Section 3. Re-format and Rate PLD**

***a. Load PLD from snowflake query***

In [46]:
#Load Doc. 6
PLDfromSnowFlake = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\Coop2024_with2023.xlsx')
#tbl_BP.head()
#tbl_BP.shape

In [47]:
#Make a copy especially when the original file is too large
tbl_BP = PLDfromSnowFlake.copy()
#Clean the empty rows/columns just in case
tbl_BP = tbl_BP.dropna(axis = 0, how = 'all')
tbl_BP = tbl_BP.dropna(axis = 1, how = 'all')
# Dropping duplicated rows
tbl_BP = tbl_BP.drop_duplicates()

In [48]:
tbl_BP.columns

Index(['CustomerID', 'CustomerName', 'Facility', 'BarrettOrderNumber',
       'Reference', 'PoNumber', 'TrackingNumber', 'ShipDate', 'Service',
       'Shipper', 'ShipToName', 'ShipToContact', 'ShipToCity', 'ShipToState',
       'ZipCode', 'ShipToCountry', 'Zone', 'Quantity', 'Dimensions',
       'ActualWeight', 'ResidentialFlag'],
      dtype='object')

In [49]:
#Required column names and type:     
# ---  ------             --------------  -----         
#1   TrackingNumber        ---------------------      int64 (or object)      
#2   ShipDate              ---------------------      datetime64[ns]
#3   Service               ---------------------      object        
#4   ZipCode               ---------------------       object
#5   Zone                  ---------------------      object          
#6   Dimensions            ---------------------      object     
#7   ActualWeight (lb as default)---------------      float64       
#8   ResidentialFlag       ---------------------      bool   
tbl_BP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44621 entries, 0 to 44620
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          44621 non-null  int64         
 1   CustomerName        44621 non-null  object        
 2   Facility            44621 non-null  object        
 3   BarrettOrderNumber  44621 non-null  object        
 4   Reference           44621 non-null  object        
 5   PoNumber            44613 non-null  object        
 6   TrackingNumber      44621 non-null  object        
 7   ShipDate            44621 non-null  datetime64[ns]
 8   Service             44621 non-null  object        
 9   Shipper             44621 non-null  object        
 10  ShipToName          44621 non-null  object        
 11  ShipToContact       1043 non-null   object        
 12  ShipToCity          44621 non-null  object        
 13  ShipToState         44621 non-null  object    

***b. Patch PLD***

In [50]:
# Patch Tracking Number
# Generating a sequence of integers to replace NaN values
sequence = iter(range(1, len(tbl_BP) + 1))

# Using list comprehension to replace NaN values directly
tbl_BP['TrackingNumber'] = [next(sequence) if pd.isna(x) else x for x in tbl_BP['TrackingNumber']]


#Patch Ship Date
# Calculating yesterday's date
yesterday_date = datetime.now() - timedelta(days=1)

# Filling null values with yesterday's date
tbl_BP['ShipDate'] = tbl_BP['ShipDate'].fillna(yesterday_date)


# Patch Zone
# Add 'FromZip' column basd on the value of Facility in the same df.
tbl_BP['FromZip'] = tbl_BP.apply(
    lambda row: 
    '02038' if row['Facility'] in ('FRA', 'MA5', 'MA6', 'BRI', 'MAN', 'MA7', 'NBO') else
    '08873' if (row['Facility'] in ('JER', 'NJ2', 'NJ3', 'NY1')) else
    '21226' if (row['Facility'] in ('GBM', 'GTZ', 'IDR')) else
    '38141' if (row['Facility'] in ('TN1', 'TN2', 'TN3', 'MEM')) else
    '38654' if (row['Facility'] in ('MS3', 'MS2', 'MS1', 'MST', 'MS4', 'MS5', 'MS6')) else
    '75041' if (row['Facility'] in ('TX1')) else
    '90640' if (row['Facility'] in ('MBC', 'CA3')) else
    '', 
    axis=1
)


# Fill "Zone" values if this column in the original PLD is insufficient
# Unify the columns types for merge and fill Zip Code to 5 digits. e.g. 01234 rather than 1234 
columns_to_fill = [(tbl_BP, 'ZipCode'), (tbl_BP, 'FromZip'), (ZipToZone, 'Origin'), (ZipToZone, 'Dest Zip')]
def zfill_column(df, col_name):
    df[col_name] = df[col_name].astype(str).str.zfill(5)

for df, col in columns_to_fill:
    zfill_column(df, col)
    
tbl_BP = tbl_BP.merge(ZipToZone[["Origin", "Dest Zip", "GNDZone"]], left_on=['FromZip', 'ZipCode'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZoneExtra'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZoneExtra'] = tbl_BP['ZoneExtra'].astype('str')
tbl_BP['ZoneExtra'] = (tbl_BP['ZoneExtra'].str.replace('44', '9') #the ZipToZone has 44, 45, and 46 as Zones. 
                              .str.replace('45', '10')
                              .str.replace('46', '11'))


# Replace 'Zone' values that are 0 or null with 'Zone_extra' values, where 'Zone_extra' is not 0 or null
tbl_BP.loc[(tbl_BP['Zone'] == 0) | tbl_BP['Zone'].isnull(), 'Zone'] = tbl_BP['ZoneExtra']
tbl_BP = tbl_BP.drop('ZoneExtra', axis=1)

#Reformat so that they can be used to merge later. 
tbl_BP['Zone'] = tbl_BP['Zone'].astype(str)
# Function to convert values
def convert_to_int(val):
    try:
        # Convert to float first to handle values like '6.0'
        return str(int(float(val)))
    except ValueError:
        # Return the original value if conversion is not possible
        return val

# Apply the function to the 'Zone' column
tbl_BP['Zone'] = tbl_BP['Zone'].apply(convert_to_int)



***c. Flag PLD (Flag instead of delete data)***

In [51]:
# Initialize 'RateFlag' column with a default value (e.g., 0)
tbl_BP['RateFlag'] = 1

# Setting 'RateFlag' to 0 for rows that meet specific conditions
tbl_BP.loc[tbl_BP.ShipDate.isnull() |
           tbl_BP.ZipCode.isnull() | 
           (tbl_BP.ZipCode == '0') |
           tbl_BP.Zone.isnull() | 
           (tbl_BP.Zone == '0') |
           tbl_BP.Dimensions.isnull() | 
           (tbl_BP.Dimensions == 0) |
           tbl_BP.ActualWeight.isnull() | 
           (tbl_BP.ActualWeight == 0) |
           (tbl_BP.ActualWeight > 150), 'RateFlag'] = 0


# Sorting by 'TrackingNumber' and then by 'ShipDate' in descending order
tbl_BP = tbl_BP.sort_values(by=['TrackingNumber', 'ShipDate'], ascending=[True, False])

# Updating 'RateFlag' for duplicates, but only where 'RateFlag' is not already 0, # For rows where 'RateFlag' is already 0, it remains unchanged
# In cases where duplicates are found, the first occurrence is considered as non-duplicate, and all subsequent occurrences are considered duplicates.
mask = tbl_BP.duplicated(subset='TrackingNumber', keep='first')     
tbl_BP.loc[mask, 'RateFlag'] = 0


# Splitting the 'Dimensions' column into three separate parts
dimensions_split = tbl_BP['Dimensions'].str.split('x', expand=True)
# Converting the split parts to numeric, errors='coerce' will handle non-numeric values
dimensions_numeric = dimensions_split.apply(pd.to_numeric, errors='coerce')
# Checking if any of the three parts are 0 or non-numeric
invalid_dimensions = dimensions_numeric.isna().any(axis=1) | (dimensions_numeric == 0).any(axis=1)
# Updating 'RateFlag' to 0 for rows with a dimension of 0
tbl_BP.loc[invalid_dimensions, 'RateFlag'] = 0
# Updating 'RateFlag' to 1 for other rows, without affecting rows already marked as 0
tbl_BP.loc[~invalid_dimensions & (tbl_BP['RateFlag'] != 0), 'RateFlag'] = 1


# List of valid values for 'Service'
valid_services = [
    "UPS Next Day Air Early", 
    "UPS Next Day Air Saver", 
    "UPS Next Day Air", 
    "UPS 2nd Day Air A.M.", 
    "UPS 2nd Day Air",
    "UPS 3 Day Select", 
    "UPS Ground", 
    "UPS SurePost 1 lb or Greater", 
    "UPS SurePost Less than 1 lb", 
    "UPS Standard", 
    "UPS Worldwide Express Saver", 
    "UPS Worldwide Express", 
    "UPS Worldwide Expedited", 
    "DHL SmartMail Parcel Expedited Max",
    "DHL SmartMail Parcel Plus Expedited", 
    "DHL SmartMail Parcel Expedited", 
    "DHL SmartMail Parcel Plus Ground", 
    "DHL SmartMail Parcel Ground", 
    "USPS 1st Class (Endicia)", 
    "USPS First Class Mail", 
    "USPS Parcel Select (Endicia)",
    "USPS Express (Endicia)", 
    "USPS Priority (Endicia)", 
    "USPS Priority Mail"
   # "UPS Mail Innovations First Class",
   # "UPS Mail Innovations Expedited"
]  # Add your valid services here

# Update 'RateFlag' to 0 for rows where 'Service' is not in the list of valid services
tbl_BP.loc[~tbl_BP['Service'].isin(valid_services), 'RateFlag'] = 0



***d. Add columns to PLD*** 

In [52]:
#turn dimensions into L W H columns.
tbl_BP[['L', 'W', 'H']] = tbl_BP['Dimensions'].str.split('x', expand=True)

#Turn float to integers using the excel way of rounding up. 
tbl_BP[['L', 'W', 'H']] = tbl_BP[['L', 'W', 'H']].apply(pd.to_numeric, errors='coerce')
tbl_BP['L'] = tbl_BP['L'].apply(lambda x: round(x,0) if pd.notna(x) and x - math.floor(x) < 0.5 else np.ceil(x) if pd.notna(x) else np.nan)
tbl_BP['W'] = tbl_BP['W'].apply(lambda x: round(x,0) if pd.notna(x) and x - math.floor(x) < 0.5 else np.ceil(x) if pd.notna(x) else np.nan)
tbl_BP['H'] = tbl_BP['H'].apply(lambda x: round(x,0) if pd.notna(x) and x - math.floor(x) < 0.5 else np.ceil(x) if pd.notna(x) else np.nan)


# Create a temporary DataFrame with the columns 'L', 'W', and 'H'
temp_df = tbl_BP[['L', 'W', 'H']]

# Sort each row from large to small, L, W, H
sorted_temp_df = np.sort(temp_df.values, axis=1)[:, ::-1]

# Assign the sorted values back to the original DataFrame
tbl_BP[['L', 'W', 'H']] = sorted_temp_df

# Create columns of GirthAndL and CubicInch
tbl_BP['GirthAndL'] = round(tbl_BP['W'] * 2 + tbl_BP['H'] * 2 + tbl_BP['L'],0)
tbl_BP['CubicInch'] = round(tbl_BP['W'] * tbl_BP['H'] * tbl_BP['L'],0)



# Add ServiceCode column to PLD
# Do not change the order of this code without knowing the logic here. 
shipping_map = {
    "UPS Next Day Air Early": "REDE",
    "UPS Next Day Air Saver": "REDS",
    "UPS Next Day Air": "RED",
    "UPS 2nd Day Air A.M.": "2DAM",
    "UPS 2nd Day Air": "BLUE",
    "UPS 3 Day Select": "ORNG",
    "UPS Ground": "GRND",
    "UPS SurePost 1 lb or Greater": "SRPT",
    "UPS SurePost Less than 1 lb": "SRPT<1",
    "UPS Standard": "CNST",
    "UPS Worldwide Express Saver": "WWXS",
    "UPS Worldwide Express Plus": "OTHER",
    "UPS Worldwide Express": "WWX",
    "UPS Worldwide Expedited": "WWE",
    "DHL SmartMail Parcel Expedited Max": "DHLEM",
    "DHL SmartMail Parcel Plus Expedited": "DHLE",
    "DHL SmartMail Parcel Expedited": "DHLE<1",
    "DHL SmartMail Parcel Plus Ground": "DHLG",
    "DHL SmartMail Parcel Ground": "DHLG<1",
    "USPS 1st Class (Endicia)": "USPSFC",
    "USPS First Class Mail": "USPSFC",
    "USPS Parcel Select (Endicia)": "USPSPS",
    "USPS Express (Endicia)": "USPSAP",
    "USPS Priority (Endicia)": "USPSAP",
    "USPS Priority Mail": "USPSAP",
    "UPS Mail Innovations First Class": "MIPH",
    "UPS Mail Innovations Expedited": "MIPLE",
}

        
def label_shipping(row):
    for key, value in shipping_map.items():
        if key in row:
            return value
    return "OTHER"

tbl_BP['ServiceCode'] = tbl_BP['Service'].apply(label_shipping)
tbl_BP['ServiceCode'] = tbl_BP.apply(lambda row: 'GRES' if (row['ResidentialFlag'] == True and row['ServiceCode'] == 'GRND') else row['ServiceCode'], axis=1)


# Add Das Code
ZipToDas['Zip'] = ZipToDas['Zip'].astype(str).apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if len(x) == 3 else x))
tbl_BP = tbl_BP.merge(ZipToDas[["Zip", "Type"]], left_on='ZipCode', right_on='Zip', how='left').rename(columns={'Type': 'DASCategory'})
tbl_BP.drop(columns=['Zip'], inplace=True)
tbl_BP.loc[~tbl_BP['ServiceCode'].isin(['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG', 'GRND', 'GRES', 'SRPT', 'SRPT<1']), 'DASCategory'] = None


# Add FSC Rate
# Specify the ServiceCodes for the three scenarios
upsAir_calculation_1 = ["REDE", "RED", "REDS", "2DAM", "BLUE", "ORNG"]  
upsGround_calculation_2 = ["GRND", "GRES", "SRPT", "SRPT<1"]
upsIntl_calculation_3 = ["WWXS", "WWX", "WWE"]
upsCan_calculation_4 = ["CNST"]
dhl_calculation_5 = ["DHLG", "DHLG<1", "DHLE", "DHLE<1", "DHLEM"]


# ups 
tbl_BP['ShipDate'] = pd.to_datetime(tbl_BP['ShipDate'])
# get unique values from column 'SHIP_DATE' and create a new dataframe. 
new_df = pd.DataFrame({'ShipDate': tbl_BP['ShipDate'].unique()})
# use this new dataframe to merge to Doc. 3 UPS section
df_merge = new_df.merge(UPS_FSC, how='cross')
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have Doc. 3 with Ship_date column to merge back to pld so that the pld can get FSC%.
df_merge = df_merge.query('ShipDate >= ShipDateStart and ShipDate <= ShipDateEnd')
# Get dhl FSC% to pld
tbl_BP = tbl_BP.merge(df_merge, on=['ShipDate'], how='left').drop(['ShipDateStart', 'ShipDateEnd'], axis=1)


# dhl 
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge = df_merge.query('ShipDate >= ShipDateStart and ShipDate <= ShipDateEnd')
tbl_BP = tbl_BP.merge(df_merge, on=['ShipDate'], how='left').drop(['ShipDateStart', 'ShipDateEnd'], axis=1)


# pick rates from the 3 FSC% columns based on ServiceCode
tbl_BP['FscRate'] = np.where(tbl_BP['ServiceCode'].isin(upsAir_calculation_1), tbl_BP['Domestic Air'], 
                         np.where(tbl_BP['ServiceCode'].isin(upsGround_calculation_2), tbl_BP['Ground'], 
                                  np.where(tbl_BP['ServiceCode'].isin(upsIntl_calculation_3), tbl_BP['International Air Export'], 
                                           np.where(tbl_BP['ServiceCode'].isin(upsCan_calculation_4), tbl_BP['International Ground Export Import'],
                                                    np.where(tbl_BP['ServiceCode'].isin(dhl_calculation_5), tbl_BP['FSC'],
                                           np.nan)))))

# Drop the original rate columns
tbl_BP = tbl_BP.drop(['Domestic Air', 'Ground', 'International Air Export', 'International Ground Export Import', 'FSC'], axis=1)



C:\Users\sliu\AppData\Local\Temp\ipykernel_21416\3559374997.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ZipToDas['Zip'] = ZipToDas['Zip'].astype(str).apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if len(x) == 3 else x))


In [53]:
# Add dim weight and actual weight rounded up column to simply the process
def cust_dimmed_weight(row):    
    if row['ServiceCode'] in ['REDE','RED','REDS','2DAM','BLUE','ORNG','GRND','GRES']:
        return np.ceil(row['CubicInch'] / dim_factor_ups)
    elif row['ServiceCode'] in ['SRPT', 'SRPT<1']:
        return np.ceil(row['CubicInch'] / dim_factor_ups_srpt)
    elif row['ServiceCode'] in ['CNST','WWX','WWXS','WWE']:
        return np.ceil(row['CubicInch'] / dim_factor_ups_intl)
    elif row['ServiceCode'] in ('DHLG', 'DHLE', 'DHLG<1', 'DHLE<1','DHLEM'):
        return np.ceil(row['CubicInch'] / dim_factor_dhl)
    elif row['ServiceCode'] in ('USPSFC', 'USPSPS', 'USPSAP'):
        return np.ceil(row['CubicInch'] / dim_factor_usps)
    elif row['ServiceCode'] in ('MIPH', 'MIPLE'):
        return np.ceil(row['CubicInch'] / dim_factor_mi)
    else:
        return np.nan  # Return NaN or an appropriate default value for other cases

# Apply the function to each row to create the new column
tbl_BP['CustomerDimWeight'] = tbl_BP.apply(cust_dimmed_weight, axis=1)

# Create Round up Columns
tbl_BP['CustomerActualWeightLb'] = np.ceil(tbl_BP['ActualWeight'])
tbl_BP['CustomerActualWeightOz'] = np.ceil(tbl_BP['ActualWeight']*16)


# Add PublishedBilledWeightLb and PublishedBilledWeightOz columns for UPS
def compute_weights(row):
    if row['ServiceCode'] in ['REDE','RED','REDS','2DAM','BLUE','ORNG','GRND','GRES','CNST','WWX','WWXS','WWE']:
        return max(row['CustomerActualWeightLb'], np.ceil(row['CubicInch']/dim_factor_ups_pub)), None
    elif row['ServiceCode'] == 'SRPT' and row['CubicInch'] >= ups_allowance:
        return max(row['CustomerActualWeightLb'], np.ceil(row['CubicInch']/dim_factor_ups_pub)), None
    elif row['ServiceCode'] == 'SRPT' and row['CubicInch'] < ups_allowance:
        return row['CustomerActualWeightLb'], None
    elif row['ServiceCode'] == 'SRPT<1' and row['CubicInch'] >= ups_allowance:
        return None, max(row['CustomerActualWeightLb'], np.ceil(row['CubicInch']/dim_factor_ups_pub))*16
    elif row['ServiceCode'] == 'SRPT<1' and row['CubicInch'] < ups_allowance:
        return None, row['CustomerActualWeightOz']
    else:
        return None, None

tbl_BP['PublishedBilledWeightLb'], tbl_BP['PublishedBilledWeightOz'] = zip(*tbl_BP.apply(compute_weights, axis=1))
tbl_BP.loc[tbl_BP['PublishedBilledWeightOz'] > 16, 'PublishedBilledWeightOz'] = np.nan



In [54]:
# Add customer billed weight. weight lb
def cust_weight_lb(row):    
    if row['ServiceCode'] in ['REDE','RED','REDS','2DAM','BLUE','ORNG','GRND','GRES','CNST','WWX','WWXS','WWE']:
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])
    elif row['ServiceCode'] == 'SRPT' and row['CubicInch'] >= ups_allowance:
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])
    elif row['ServiceCode'] == 'SRPT' and row['CubicInch'] < ups_allowance:
        return row['CustomerActualWeightLb']
    elif row['ServiceCode'] in ['DHLG', 'DHLE'] and (row['CubicInch'] >= dhl_allowance and row['ActualWeight'] >= 1):
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])
    elif row['ServiceCode'] in ['DHLG', 'DHLE'] and (row['CubicInch'] < dhl_allowance or row['ActualWeight'] < 1):
        return row['CustomerActualWeightLb']
    elif row['ServiceCode'] == 'USPSPS' and row['CubicInch'] > usps_allowance:
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])
    elif row['ServiceCode'] == 'USPSPS' and row['CubicInch'] <= usps_allowance:
        return row['CustomerActualWeightLb']
    elif row['ServiceCode'] == 'MIPH' and (row['CubicInch'] > mi_allowance and row['ActualWeight'] >= 1):
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])
    elif row['ServiceCode'] == 'MIPH' and (row['CubicInch'] <= mi_allowance or row['ActualWeight'] < 1):
        return row['CustomerActualWeightLb']
    elif row['ServiceCode'] == 'DHLEM' and (row['CubicInch'] >= dhl_allowance and row['ActualWeight'] >= 1):
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])
    elif row['ServiceCode'] == 'DHLEM' and (row['CubicInch'] < dhl_allowance or row['ActualWeight'] < 1) and row['CustomerActualWeightOz'] >= 16:
        return row['CustomerActualWeightLb']
    elif row['ServiceCode'] == 'USPSAP' and (row['CubicInch'] > usps_allowance):
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])
    elif row['ServiceCode'] == 'USPSAP' and (row['CubicInch'] <= usps_allowance) and row['CustomerActualWeightOz'] > 8:
        return row['CustomerActualWeightLb']
    elif row['ServiceCode'] == 'USPSAP' and (row['CubicInch'] <= usps_allowance) and row['CustomerActualWeightOz'] <= 8:
        return 0.5
    
# weight oz
def cust_weight_oz(row):    
    if row['ServiceCode'] == 'SRPT<1' and row['CubicInch'] >= ups_allowance:
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])*16
    if row['ServiceCode'] == 'SRPT<1' and row['CubicInch'] < ups_allowance:
        return row['CustomerActualWeightOz']
    elif row['ServiceCode'] in ['DHLG<1', 'DHLE<1'] and (row['CubicInch'] >= dhl_allowance and row['ActualWeight'] >= 1):
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])*16
    elif row['ServiceCode'] in ['DHLG<1', 'DHLE<1'] and (row['CubicInch'] < dhl_allowance or row['ActualWeight'] < 1):
        return row['CustomerActualWeightOz']
    elif row['ServiceCode'] == 'USPSFC' and row['CubicInch'] > usps_allowance:
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])*16
    elif row['ServiceCode'] == 'USPSFC' and row['CubicInch'] <= usps_allowance:
        return row['CustomerActualWeightOz']
    elif row['ServiceCode'] == 'MIPLE' and (row['CubicInch'] > mi_allowance and row['ActualWeight'] >= 1):
        return max(row['CustomerActualWeightLb'], row['CustomerDimWeight'])*16
    elif row['ServiceCode'] == 'MIPLE' and (row['CubicInch'] <= mi_allowance or row['ActualWeight'] < 1):
        return row['CustomerActualWeightOz']
    elif row['ServiceCode'] == 'DHLEM' and (row['CubicInch'] < dhl_allowance or row['ActualWeight'] < 1) and row['CustomerActualWeightOz'] < 16:
        return row['CustomerActualWeightOz']     
    
tbl_BP['CustomerBilledWeightLb'] = tbl_BP.apply(cust_weight_lb, axis=1)
tbl_BP['CustomerBilledWeightOz'] = tbl_BP.apply(cust_weight_oz, axis=1)
tbl_BP = tbl_BP.drop(['CustomerDimWeight', 'CustomerActualWeightLb', 'CustomerActualWeightOz'], axis=1)
tbl_BP.loc[tbl_BP['CustomerBilledWeightOz'] > 16, 'CustomerBilledWeightOz'] = np.nan
#Unqualified service cases:
#If billweight > 1, 'SRPT<1', 'DHLG<1', 'DHLE<1', and 'USPSFC_CPP' are not eligible. These packages will not be rated.   

tbl_BP['PublishedBilledWeightOz'] = tbl_BP['PublishedBilledWeightOz'].astype(float)
tbl_BP['PublishedBilledWeightLb'] = tbl_BP['PublishedBilledWeightLb'].astype(float)
tbl_BP['CustomerBilledWeightOz'] = tbl_BP['CustomerBilledWeightOz'].astype(float)
tbl_BP['CustomerBilledWeightLb'] = tbl_BP['CustomerBilledWeightLb'].astype(float)



***e. Add Freight Rates (Published Freight "UPS only" & Barrett Freight "except UPS") to PLD***

In [55]:
# Add Freight Rates to PLD. 

# Published Rates & Lb
group_pr_lb = ['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG', 'GRND', 'GRES', 'SRPT', 'CNST', 'WWX', 'WWXS', 'WWE'] 
# Published Rates & Oz
group_pr_oz = ['SRPT<1']  
# Barrett Rates & Lb
group_br_lb = ['MIPH', 'USPSPS', 'USPSAP']  
# Barrett Rates & Oz
group_br_oz = ['MIPLE', 'USPSFC'] 
#Barrett Rate & Lb, DHL. separate because DHL needs additional merge to Facility.
group_br_lb_dhl = ['DHLG', 'DHLE', 'DHLEM'] 
#Barrett Rate & Oz, DHL
group_br_oz_dhl = ['DHLG<1', 'DHLE<1', 'DHLEM']  



In [56]:
# 1. Merge for Published Rates & Lb
tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['PublishedRate'].notna() & FreightCharge['ServiceCode'].isin(group_pr_lb)][['ServiceCode', 'WeightLb', 'Zone', 'PublishedRate']],
    left_on=['ServiceCode', 'PublishedBilledWeightLb', 'Zone'],
    right_on=['ServiceCode', 'WeightLb', 'Zone'],
    how='left'
).rename(columns={'PublishedRate': 'PublishedRate_Lb'}).drop(columns='WeightLb')


# 2. Merge for Published Rates & Oz
tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['PublishedRate'].notna() & FreightCharge['ServiceCode'].isin(group_pr_oz)][['ServiceCode', 'WeightOz', 'Zone', 'PublishedRate']],
    left_on=['ServiceCode', 'PublishedBilledWeightOz', 'Zone'],
    right_on=['ServiceCode', 'WeightOz', 'Zone'],
    how='left'
).rename(columns={'PublishedRate': 'PublishedRate_Oz'}).drop(columns='WeightOz')


# 3. Merge for Barrett Rates & Lb
tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['BarrettRate'].notna() & FreightCharge['ServiceCode'].isin(group_br_lb)][['ServiceCode', 'WeightLb', 'Zone', 'BarrettRate']],
    left_on=['ServiceCode', 'CustomerBilledWeightLb', 'Zone'],
    right_on=['ServiceCode', 'WeightLb', 'Zone'],
    how='left'
).rename(columns={'BarrettRate': 'BarrettRate_Lb'}).drop(columns='WeightLb')


# 4. Merge for Barrett Rates & Oz
tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['BarrettRate'].notna() & FreightCharge['ServiceCode'].isin(group_br_oz)][['ServiceCode', 'WeightOz', 'Zone', 'BarrettRate']],
    left_on=['ServiceCode', 'CustomerBilledWeightOz', 'Zone'],
    right_on=['ServiceCode', 'WeightOz', 'Zone'],
    how='left'
).rename(columns={'BarrettRate': 'BarrettRate_Oz'}).drop(columns='WeightOz')

# 5. Merge for Barrett Rate & Lb, DHL
# Define the mask to select rows where ServiceCode starts with 'DHL'
mask = FreightCharge['ServiceCode'].str.startswith('DHL')
FreightCharge[['ServiceCode_DHL', 'Facility_DHL']] = FreightCharge.loc[mask, 'ServiceCode'].str.extract(r'(DHLG<1|DHLE<1|DHLG|DHLE|DHLEM)_([0-9]{7})')
key_mapping = {
    'FRA': '5114358', 'MA5': '5114358', 'MA6': '5114358', 'BRI': '5123282', 'MAN': '5123280', 'MA7': '5114358', 'NBO': '5114358', 
    'JER': '5122444', 'NJ2': '5122444', 'NJ3': '5122444', 'NY1': '5122444', 'GBM': '5123283', 'GTZ': '5123283', 'IDR': '5123283', 
    'TN1': '5122893', 'TN2': '5122893', 'TN3': '5122893', 'MEM': '5122893', 'MS3': '5122723', 'MS2': '5122723', 'MS1': '5122723', 
    'MST': '5122723', 'MS4': '5122723', 'MS5': '5122723', 'TX1': '5122855', 'MBC': '5123556'
}

tbl_BP['key_mapped'] = tbl_BP['Facility'].map(key_mapping)

tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['ServiceCode_DHL'].notna() & FreightCharge['ServiceCode_DHL'].isin(group_br_lb_dhl)][['ServiceCode_DHL', 'Facility_DHL', 'WeightLb', 'Zone', 'BarrettRate']],
    left_on=['ServiceCode', 'key_mapped', 'CustomerBilledWeightLb', 'Zone'],
    right_on=['ServiceCode_DHL', 'Facility_DHL', 'WeightLb', 'Zone'],
    how='left'
).rename(columns={'BarrettRate': 'BarrettRate_Lb_DHL'}).drop(columns=['ServiceCode_DHL', 'Facility_DHL', 'WeightLb'])


# 6. Merge for Barrett Rate & Oz, DHL
tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['ServiceCode_DHL'].notna() & FreightCharge['ServiceCode_DHL'].isin(group_br_oz_dhl)][['ServiceCode_DHL', 'Facility_DHL', 'WeightOz', 'Zone', 'BarrettRate']],
    left_on=['ServiceCode', 'key_mapped', 'CustomerBilledWeightOz', 'Zone'],
    right_on=['ServiceCode_DHL', 'Facility_DHL', 'WeightOz', 'Zone'],
    how='left'
).rename(columns={'BarrettRate': 'BarrettRate_Oz_DHL'}).drop(columns=['ServiceCode_DHL', 'Facility_DHL', 'WeightOz', 'key_mapped'])


# Special case. Set the 'BarrettRate_Lb' values to NaN for rows that meet this condition as we want to use oversize rate only.
mask_1 = (tbl_BP['ServiceCode'] == 'USPSPS') & (tbl_BP['GirthAndL'] > 108) & (tbl_BP['GirthAndL'] <= 130)
mask_2 = (tbl_BP['ServiceCode'] == 'USPSPS') & (tbl_BP['GirthAndL'] > 130)
tbl_BP.loc[mask_1, 'BarrettRate_Lb'] = np.nan
tbl_BP.loc[mask_2, 'BarrettRate_Lb'] = np.nan

#uspsps. Use the mask to merge only those rows directly in tbl_BP
merged_subset = tbl_BP[mask_1].merge(
    FreightCharge[(FreightCharge['ServiceCode'] == 'USPSPS') & (FreightCharge['WeightLb'] == 9999)][['ServiceCode', 'Zone', 'BarrettRate']],
    left_on=['ServiceCode', 'Zone'],
    right_on=['ServiceCode', 'Zone'],
    how='left'
)

# Then assign the 'BarrettRate' values from the merged subset back to 'BarrettRate_Lb' in the original tbl_BP where the mask is True
tbl_BP.loc[mask_1, 'BarrettRate_Lb'] = merged_subset['BarrettRate']


# Combine all the Rate columns into one 'Rate' column
tbl_BP['PublishedFreight'] = tbl_BP['PublishedRate_Lb'].combine_first(tbl_BP['PublishedRate_Oz'])
tbl_BP['BarrettFreight'] = tbl_BP['BarrettRate_Lb'].combine_first(tbl_BP['BarrettRate_Oz']).combine_first(tbl_BP['BarrettRate_Lb_DHL']).combine_first(tbl_BP['BarrettRate_Oz_DHL'])

# Drop the intermediate Rate columns
columns_to_drop = ['PublishedRate_Lb', 'PublishedRate_Oz', 'BarrettRate_Lb', 'BarrettRate_Oz', 'BarrettRate_Lb_DHL', 'BarrettRate_Oz_DHL']
tbl_BP.drop(columns=columns_to_drop, inplace=True)


# Filter rows where 'exclude_rates' column value is in the provided list
condition = ~tbl_BP['ServiceCode'].isin(["REDE", "RED", "REDS", "2DAM", "BLUE", "ORNG", "GRND", "GRES", "SRPT", "SRPT<1", "CNST", "WWX", "WWXS", "WWE"])

# If the condition is met, then set 'PublishedFreight' to NaN because they are actually Barrett Rate here if not UPS 
tbl_BP.loc[condition, 'PublishedFreight'] = np.nan



***f. Add Customer Freight to PLD (and Barrett Freight "UPS only" to PLD)***

In [57]:
# Add UPS Pub Rates from the same rate card location to PLD only for UPS Customer Rate calculation. 

# temp Published Rates & Lb
group_pr_lb = ['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG', 'GRND', 'GRES', 'SRPT', 'CNST', 'WWX', 'WWXS', 'WWE'] 

# temp Published Rates & Oz
group_pr_oz = ['SRPT<1']  


# 1. Merge for Published Rates & Lb
tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['PublishedRate'].notna() & FreightCharge['ServiceCode'].isin(group_pr_lb)][['ServiceCode', 'WeightLb', 'Zone', 'PublishedRate']],
    left_on=['ServiceCode', 'CustomerBilledWeightLb', 'Zone'],
    right_on=['ServiceCode', 'WeightLb', 'Zone'],
    how='left'
).rename(columns={'PublishedRate': 'PublishedRate_Lb'}).drop(columns='WeightLb')


# 2. Merge for Published Rates & Oz
tbl_BP = tbl_BP.merge(
    FreightCharge[FreightCharge['PublishedRate'].notna() & FreightCharge['ServiceCode'].isin(group_pr_oz)][['ServiceCode', 'WeightOz', 'Zone', 'PublishedRate']],
    left_on=['ServiceCode', 'CustomerBilledWeightOz', 'Zone'],
    right_on=['ServiceCode', 'WeightOz', 'Zone'],
    how='left'
).rename(columns={'PublishedRate': 'PublishedRate_Oz'}).drop(columns='WeightOz')


# Combine all the Rate columns into one 'Rate' column
tbl_BP['PublishedFreightTemp'] = tbl_BP['PublishedRate_Lb'].combine_first(tbl_BP['PublishedRate_Oz'])

# Drop the intermediate Rate columns
columns_to_drop = ['PublishedRate_Lb', 'PublishedRate_Oz']
tbl_BP.drop(columns=columns_to_drop, inplace=True)


In [58]:
# Specify the ServiceCodes for the 4 different ways of calculating client rates. See the table above.  
group_disc_cr = ['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG']  
group_wtbreaklb_disc_cr = ['GRND', 'GRES', 'SRPT']
group_wtbreakoz_disc_cr = ['SRPT<1']
group_marg_cr = ['DHLG', 'DHLG<1', 'DHLE', 'DHLE<1', 'DHLEM', 'USPSPS', 'USPSFC', 'USPSAP']
group_intl_disc_cr = ['CNST', 'WWX', 'WWXS', 'WWE']

# Use a regular expression pattern to extract the minimum and maximum weights directly into 'min_weight' and 'max_weight' for merge
pattern = r'(\d+)(?:-(\d+))?(?:lb|oz)?\+?'

# Extract and assign the min_weight and max_weight
ClientCharge[['min_weight', 'max_weight']] = ClientCharge['ServiceNote'].str.extract(pattern)

# Dictionary to map ServiceNote values to max_weight
service_note_weight = {
    '31+lb': '999',
    '9lb': '9',
    '200+lb': '999',
    '10+lb': '999',
    '11+lb': '999'
}

# Loop through the dictionary and apply the mappings
for service_note, max_weight in service_note_weight.items():
    mask = ClientCharge['ServiceNote'] == service_note
    ClientCharge.loc[mask, 'max_weight'] = max_weight

# Convert min_weight and max_weight to integers, keeping NaN values intact
ClientCharge['min_weight'] = ClientCharge['min_weight'].astype(float).astype('Int64')
ClientCharge['max_weight'] = ClientCharge['max_weight'].astype(float).astype('Int64')


condition = ClientCharge['ServiceNote'].isin(['<=5lb&Zone8', 'Pkg', 'ToCanada', 'SatDelivery']) 
# Apply the condition to set Column2 and Column3 to NaN where the condition is True
ClientCharge.loc[condition, ['min_weight', 'max_weight']] = np.nan


# Filter out rows where PriceType is 'Special' as we will handle Special cases at the end. 
filtered_ClientCharge = ClientCharge[ClientCharge['PriceType'] != 'Special']

# Change data type of 'Price' column to float
filtered_ClientCharge['Price'] = filtered_ClientCharge['Price'].astype(float)
filtered_ClientCharge['PriceMin'] = pd.to_numeric(filtered_ClientCharge['PriceMin'], errors='coerce')
filtered_ClientCharge['PriceMin'] = filtered_ClientCharge['PriceMin'].astype(float)



C:\Users\sliu\AppData\Local\Temp\ipykernel_21416\27793880.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ClientCharge[['min_weight', 'max_weight']] = ClientCharge['ServiceNote'].str.extract(pattern)
C:\Users\sliu\AppData\Local\Temp\ipykernel_21416\27793880.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ClientCharge[['min_weight', 'max_weight']] = ClientCharge['ServiceNote'].str.extract(pattern)
C:\Users\sliu\AppData\Local\Temp\ipykernel_21416\27793880.py:29: SettingWithCopyWarning: 
A value is t

In [59]:
# Merge `PLD` with `ClientCharge`. Due to duplications for clientRate_calculation_2 groups we save to a new df.  
merged_df = tbl_BP.merge(filtered_ClientCharge[['CompanyID', 'ServiceCode', 'ServiceNote', 'PriceMin', 'Price', 'min_weight', 'max_weight']], 
                         left_on=['CustomerID', 'ServiceCode'], 
                         right_on=['CompanyID', 'ServiceCode'], 
                         how='left')


# Calculate client rate for group_disc
mask1 = merged_df['ServiceCode'].isin(group_disc_cr)

merged_df.loc[mask1, 'CustomerFreight'] = np.nanmax(
    [
        merged_df.loc[mask1, 'PublishedFreightTemp'] * (1 - merged_df.loc[mask1, 'Price']),
        merged_df.loc[mask1, 'PriceMin']
    ],
    axis=0
).round(2)


# Calculate client rate for clientRate_calculation_2
mask2 = (
    merged_df['ServiceCode'].isin(group_wtbreaklb_disc_cr) &
    (merged_df['min_weight'] <= merged_df['CustomerBilledWeightLb']) &
    (merged_df['max_weight'] >= merged_df['CustomerBilledWeightLb'])
)

merged_df.loc[mask2, 'CustomerFreight'] = np.nanmax(
    [
        merged_df.loc[mask2, 'PublishedFreightTemp'] * (1 - merged_df.loc[mask2, 'Price']),
        merged_df.loc[mask2, 'PriceMin']
    ],
    axis=0
).round(2)


# Modify the condition for dropping rows
# We want to drop rows where 'CustomerFreight' is null AND the weight condition is not met
mask_drop = (
    merged_df['ServiceCode'].isin(group_wtbreaklb_disc_cr) & 
    merged_df['CustomerFreight'].isna() &
    ~((merged_df['min_weight'] <= merged_df['CustomerBilledWeightLb']) &
      (merged_df['max_weight'] >= merged_df['CustomerBilledWeightLb']))
)

merged_df = merged_df[~mask_drop]


# Calculate client rate for clientRate_calculation_3
mask3 = (
    merged_df['ServiceCode'].isin(group_wtbreakoz_disc_cr) &
    (merged_df['min_weight'] <= merged_df['CustomerBilledWeightOz']) &
    (merged_df['max_weight'] >= merged_df['CustomerBilledWeightOz'])
)

merged_df.loc[mask3, 'CustomerFreight'] = np.nanmax(
    [
        merged_df.loc[mask3, 'PublishedFreightTemp'] * (1 - merged_df.loc[mask3, 'Price']),
        merged_df.loc[mask3, 'PriceMin']
    ],
    axis=0
).round(2)


# Calculate client rate for clientRate_calculation_4
mask4 = merged_df['ServiceCode'].isin(group_marg_cr)
merged_df.loc[mask4, 'CustomerFreight'] = round(merged_df.loc[mask4, 'BarrettFreight'] / (1 - merged_df.loc[mask4, 'Price']),2)


# International has min for every Zone. 
merged_df['min_intl_cr'] = merged_df['Zone'].map(mapping_min_br_intl) + 4


# Calculate client rate for group_disc_5

mask5 = merged_df['ServiceCode'].isin(group_intl_disc_cr)

merged_df.loc[mask5, 'CustomerFreight'] = np.nanmax(
    [
        merged_df.loc[mask5, 'PublishedFreight'] * (1 - merged_df.loc[mask5, 'Price']),
        merged_df.loc[mask5, 'min_intl_cr']
    ],
    axis=0
).round(2)


# Define the list of specific services
all_specific_services = group_disc_cr + group_wtbreaklb_disc_cr + group_wtbreakoz_disc_cr + group_intl_disc_cr

# Create masks for your conditions
mask_pub_freight_null = merged_df['PublishedFreightTemp'].isnull()
mask_price_null = merged_df['Price'].isnull()
mask_service_in_list = merged_df['Service'].isin(all_specific_services)

# Apply the condition: Set 'CustomerFreight' to NaN only when the service is in the specific list
merged_df.loc[(mask_pub_freight_null | mask_price_null) & mask_service_in_list, 'CustomerFreight'] = np.nan


# attached the CustomerFreight back to the original PLD
tbl_BP = tbl_BP.merge(
    merged_df[['TrackingNumber', 'CustomerFreight']],
    left_on=['TrackingNumber'],
    right_on=['TrackingNumber'],
    how='left'
)


In [60]:
# Add UPS Barrett Freight

# Specify the ServiceCodes for the 4 different ways of calculating client rates. See the table above.  
group_disc_br = ['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG']  
group_wtbreaklb_disc_br = ['GRND', 'GRES', 'SRPT']
group_wtbreakoz_disc_br = ['SRPT<1']
group_min_br = ['CNST', 'WWX', 'WWXS', 'WWE']


# Merge `PLD` with `ClientCharge`. Note that there will be duplications for clientRate_calculation_2 groups.  
merged_df = tbl_BP.merge(filtered_ClientCharge[filtered_ClientCharge['Company'] == 'Barrett Distribution'][['ServiceCode', 'ServiceNote', 'PriceMin', 'Price', 'min_weight', 'max_weight']], 
                         left_on=['ServiceCode'], 
                         right_on=['ServiceCode'], 
                         how='left')


# Calculate Barrett rate for group_disc
mask5 = merged_df['ServiceCode'].isin(group_disc_br)

merged_df.loc[mask5, 'BarrettFreight'] = np.nanmax(
    [
        merged_df.loc[mask5, 'PublishedFreightTemp'] * (1 - merged_df.loc[mask5, 'Price']),
        merged_df.loc[mask5, 'PriceMin']
    ],
    axis=0
).round(2)


# Calculate Barrett rate for clientRate_calculation_2
mask6 = (
    merged_df['ServiceCode'].isin(group_wtbreaklb_disc_br) &
    (merged_df['min_weight'] <= merged_df['CustomerBilledWeightLb']) &
    (merged_df['max_weight'] >= merged_df['CustomerBilledWeightLb'])
)


merged_df.loc[mask6, 'BarrettFreight'] = np.nanmax(
    [
        merged_df.loc[mask6, 'PublishedFreightTemp'] * (1 - merged_df.loc[mask6, 'Price']),
        merged_df.loc[mask6, 'PriceMin']
    ],
    axis=0
).round(2)


# Delete the duplications occurred in merge above only for clientRate_calculation_2
mask_drop = (
    merged_df['ServiceCode'].isin(group_wtbreaklb_disc_br) & 
    merged_df['BarrettFreight'].isna() &
    ~((merged_df['min_weight'] <= merged_df['CustomerBilledWeightLb']) &
      (merged_df['max_weight'] >= merged_df['CustomerBilledWeightLb']))
)

merged_df = merged_df[~mask_drop]


# Calculate Barrett rate for clientRate_calculation_3
mask7 = (
    merged_df['ServiceCode'].isin(group_wtbreakoz_disc_br) &
    (merged_df['min_weight'] <= merged_df['CustomerBilledWeightOz']) &
    (merged_df['max_weight'] >= merged_df['CustomerBilledWeightOz'])
)

merged_df.loc[mask7, 'BarrettFreight'] = np.nanmax(
    [
        merged_df.loc[mask7, 'PublishedFreight'] * (1 - merged_df.loc[mask7, 'Price']),
        merged_df.loc[mask7, 'PriceMin']
    ],
    axis=0
).round(2)



merged_df['min_intl'] = merged_df['Zone'].map(mapping_min_br_intl)

mask8 = merged_df['ServiceCode'].isin(group_min_br)

merged_df.loc[mask8, 'BarrettFreight'] = np.nanmax(
    [
        merged_df.loc[mask8, 'PublishedFreightTemp'] * (1 - merged_df.loc[mask8, 'Price']),
        merged_df.loc[mask8, 'min_intl']
    ],
    axis=0
).round(2)


# attached the BarrettFreight back to the original PLD
tbl_BP = tbl_BP.merge(
    merged_df[['TrackingNumber', 'BarrettFreight']],
    left_on=['TrackingNumber'],
    right_on=['TrackingNumber'],
    how='left', suffixes=('', '_merged_df')
)


# Replace the NaNs in 'rate' with values from 'rate_merged_df'
tbl_BP['BarrettFreight'] = tbl_BP['BarrettFreight'].combine_first(tbl_BP['BarrettFreight_merged_df'])

# Drop the 'rate_merged_df' column
tbl_BP = tbl_BP.drop('BarrettFreight_merged_df', axis=1)



***g. Add UPS Published & Barrett & Customer Resi, Das, and Ahs fee***

In [61]:
# Add Resi Code
tbl_BP['ResCode'] = tbl_BP.apply(lambda row:
                                        'RESA' if (row['ServiceCode'] in ('REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG') and (row['ResidentialFlag'] == True)) else
                                        ('RESG' if (row['ServiceCode'] == 'GRES' and row['ResidentialFlag'] == True) else ('')), axis=1)



# Add Das Code
tbl_BP['DasCode'] = tbl_BP.apply(
    lambda row: 
    ('DASCG' if (row['ServiceCode'] == 'GRND' and row['ResidentialFlag'] == False and row['DASCategory'] == 'DAS') else
    ('DASECG' if (row['ServiceCode'] == 'GRND' and row['ResidentialFlag'] == False and row['DASCategory'] == 'DASE') else
    ('DASRG' if (row['ServiceCode'] == 'GRES' and row['ResidentialFlag'] == True and row['DASCategory'] == 'DAS') else
    ('DASERG' if (row['ServiceCode'] == 'GRES' and row['ResidentialFlag'] == True and row['DASCategory'] == 'DASE') else
    ('DASCA' if (row['ServiceCode'] in ('REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG') and row['ResidentialFlag'] == False and row['DASCategory'] == 'DAS') else
    ('DASECA' if (row['ServiceCode'] in ('REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG') and row['ResidentialFlag'] == False and row['DASCategory'] == 'DASE') else
    ('DASRA' if (row['ServiceCode'] in ('REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG') and row['ResidentialFlag'] == True and row['DASCategory'] == 'DAS') else
    ('DASERA' if (row['ServiceCode'] in ('REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG') and row['ResidentialFlag'] == True and row['DASCategory'] == 'DASE') else
    ('DASSP' if (row['ServiceCode'] in ('SRPT', 'SRPT<1') and row['DASCategory'] == 'DAS') else
    ('DASESP' if (row['ServiceCode'] in ('SRPT', 'SRPT<1') and row['DASCategory'] == 'DASE') else
    ('REM' if (row['DASCategory'] == 'RA') else
    ('REMAK' if (row['DASCategory'] == 'AK') else
    ('REMHI' if (row['DASCategory'] == 'HI') else
    ('')))))))))))))),  # This is your catch-all condition
    axis=1)



# Add 4 AhsCodes separately, handle international separately
target_servicecode = ['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG', 'GRND', 'GRES', 'SRPT', 'SRPT<1']

# Define a function to generate AhsCode values based on conditions
def generate_ahs_code(row, column_name):
    if row['ServiceCode'] not in target_servicecode:
        return None

    if column_name == 'Ahs1Code':
        if row['GirthAndL'] > 105:
            if row['Zone'] == '2':
                return 'AHG2'
            elif row['Zone'] in ('3','4'):
                return 'AHG34'
            elif row['Zone'] in ('5','6','7','8','9','10','11'):
                return 'AHG5'
    
    elif column_name == 'Ahs2Code':
        if row['L'] > 48:
            if row['Zone'] == '2':
                return 'AHL2'
            elif row['Zone'] in ('3','4'):
                return 'AHL34'
            elif row['Zone'] in ('5','6','7','8','9','10','11'):
                return 'AHL5'

    elif column_name == 'Ahs3Code':
        if row['W'] > 30:
            if row['Zone'] == '2':
                return 'AHW2'
            elif row['Zone'] in ('3','4'):
                return 'AHW34'
            elif row['Zone'] in ('5','6','7','8','9','10','11'):
                return 'AHW5'

    elif column_name == 'Ahs4Code':
        if row['ActualWeight'] > 50:
            if row['Zone'] == '2':
                return 'AHLB2'
            elif row['Zone'] in ('3','4'):
                return 'AHLB34'
            elif row['Zone'] in ('5','6','7','8','9','10','11'):
                return 'AHLB5'

    return None

# Generate AhsCode columns
for i in range(1, 5):
    column_name = f'Ahs{i}Code'
    tbl_BP[column_name] = tbl_BP.apply(lambda row: generate_ahs_code(row, column_name), axis=1)
 


 # Get Published Rates. 
# Need to temporarily change the column name to prevent duplicated columns from confusing the merge function
tbl_BP = tbl_BP.rename(columns={'ServiceCode': 'TempServiceCode'})

# Define service code lists
service_codes = {
    'Res': ['RESA', 'RESG'],
    'Das': ['DASCG', 'DASECG', 'DASRG', 'DASERG', 'DASCA', 'DASECA', 'DASRA', 'DASERA', 'DASSP', 'DASESP', 'REM', 'REMAK', 'REMHI'],
    'Ahs1': ['AHG2', 'AHG34', 'AHG5'],
    'Ahs2': ['AHL2', 'AHL34', 'AHL5'],
    'Ahs3': ['AHW2', 'AHW34', 'AHW5'],
    'Ahs4': ['AHLB2', 'AHLB34', 'AHLB5']
}


# Merge service codes into tbl_BP. Merge multiple columns together. 
for code_type, service_list in service_codes.items():
    column_name = f'Published{code_type}'
    tbl_BP = tbl_BP.merge(
        ClientCharge[(ClientCharge['Company'] == 'UPS') & (ClientCharge['ServiceCode'].isin(service_list))][['ServiceCode', 'Price']],
        left_on=[f'{code_type.capitalize()}Code'],
        right_on=['ServiceCode'],
        how='left'
    ).rename(columns={'Price': column_name}).drop(columns=['ServiceCode'])




In [62]:
# Get Client discounts
for code_type, service_list in service_codes.items():
    code_column_name = f'{code_type}Code'
    tbl_BP = tbl_BP.merge(
        ClientCharge[(ClientCharge['ServiceCode'].isin(service_list))][['CompanyID', 'ServiceCode', 'Price']],
        left_on=['CustomerID', code_column_name],
        right_on=['CompanyID', 'ServiceCode'],
        how='left'
    ).rename(columns={'Price': f'Discount{code_type}'}).drop(columns=['CompanyID', 'ServiceCode'])

    
# Get Client Rates. Fill empty spaces as 0 for the convenience of calculation. 
cols_to_fill = ['PublishedRes', 'PublishedDas', 'PublishedAhs1', 'PublishedAhs2', 'PublishedAhs3', 'PublishedAhs4',
                'DiscountRes', 'DiscountDas', 'DiscountAhs1', 'DiscountAhs2', 'DiscountAhs3', 'DiscountAhs4']

# Fill NaN in the specified columns with 0 for easier calculation later
tbl_BP[cols_to_fill] = tbl_BP[cols_to_fill].fillna(0)


# Calculate for Customer Resi, Das, and Ahs rates
suffixes = ['Res', 'Das', 'Ahs1', 'Ahs2', 'Ahs3', 'Ahs4']


# Looping through each suffix to perform the calculations
for suffix in suffixes:
    published_col = f'Published{suffix}'
    discount_col = f'Discount{suffix}'
    customer_col = f'Customer{suffix}'

    tbl_BP[customer_col] = round(tbl_BP[published_col] * (1 - tbl_BP[discount_col]), 2)


    #Get Client discounts
for code_type, service_list in service_codes.items():
    code_column_name = f'{code_type}Code'
    tbl_BP = tbl_BP.merge(
        ClientCharge[(ClientCharge['Company'] == 'Barrett Distribution') & (ClientCharge['ServiceCode'].isin(service_list))][['ServiceCode', 'Price']],
        left_on=[code_column_name],
        right_on=['ServiceCode'],
        how='left'
    ).rename(columns={'Price': f'BarrettDiscount{code_type}'}).drop(columns=['ServiceCode'])

    

# Get Client Rates. Fill empty spaces as 0 for the convenience of calculation. 
cols_to_fill = ['BarrettDiscountRes', 'BarrettDiscountDas', 'BarrettDiscountAhs1', 'BarrettDiscountAhs2', 
                'BarrettDiscountAhs3', 'BarrettDiscountAhs4']

# Fill NaN in the specified columns with 0 for easier calculation later
tbl_BP[cols_to_fill] = tbl_BP[cols_to_fill].fillna(0)


# Calculate for Customer Resi, Das, and Ahs rates
suffixes = ['Res', 'Das', 'Ahs1', 'Ahs2', 'Ahs3', 'Ahs4']


# Looping through each suffix to perform the calculations
for suffix in suffixes:
    published_col = f'Published{suffix}'
    barrett_discount_col = f'BarrettDiscount{suffix}'
    barrett_col = f'Barrett{suffix}'

    tbl_BP[barrett_col] = round(tbl_BP[published_col] * (1 - tbl_BP[barrett_discount_col]), 2)


tbl_BP['PublishedAhs'] = tbl_BP['PublishedAhs1'] + tbl_BP['PublishedAhs2'] + tbl_BP['PublishedAhs3'] + tbl_BP['PublishedAhs4']
tbl_BP['CustomerAhs'] = tbl_BP['CustomerAhs1'] + tbl_BP['CustomerAhs2'] + tbl_BP['CustomerAhs3'] + tbl_BP['CustomerAhs4']
tbl_BP['BarrettAhs'] = tbl_BP['BarrettAhs1'] + tbl_BP['BarrettAhs2'] + tbl_BP['BarrettAhs3'] + tbl_BP['BarrettAhs4']


# Drop the intermediate Rate columns
columns_to_drop = ['ResCode', 'DasCode', 'Ahs1Code', 'Ahs2Code', 'Ahs3Code', 'Ahs4Code', 
                   'DiscountRes', 'DiscountDas', 'DiscountAhs1', 'DiscountAhs2', 'DiscountAhs3', 'DiscountAhs4', 
                   'BarrettDiscountRes', 'BarrettDiscountDas', 'BarrettDiscountAhs1', 'BarrettDiscountAhs2', 
                   'BarrettDiscountAhs3', 'BarrettDiscountAhs4', 
                   'PublishedAhs1', 'PublishedAhs2', 'PublishedAhs3', 'PublishedAhs4', 
                   'BarrettAhs1', 'BarrettAhs2', 'BarrettAhs3', 'BarrettAhs4',
                   'CustomerAhs1', 'CustomerAhs2', 'CustomerAhs3', 'CustomerAhs4']

tbl_BP.drop(columns=columns_to_drop, inplace=True)

# Name the column back to its original name
tbl_BP = tbl_BP.rename(columns={'TempServiceCode': 'ServiceCode'})
    


***h. Add dhl & usps accessorials***

In [63]:
#dhl_cubic_inch_threshold_1 = 1728/ dhl_cubic_inch_threshold_2 = 3456/ dhl_weight_threshold = 1 
#dhl_g_and_l_threshold_1 = 50/ dhl_g_and_l_threshold_2 = 84
#dhl_max_l_threshold_1 = 22/ dhl_max_l_threshold_2 = 27/ dhl_max_l_threshold_3 = 30
#dates = ['2023-01-01', '2023-09-30', '2023-10-01', '2023-12-31']
#nqd_dhl_date_1, nqd_dhl_date_2, nqd_dhl_date_3, nqd_dhl_date_4 = [pd.to_datetime(date) for date in dates]

# Mask for specific ServiceCode values
service_codes = ["DHLG", "DHLG<1", "DHLE", "DHLE<1", "DHLEM"]
mask_dhl = tbl_BP['ServiceCode'].isin(service_codes)

# Apply transformations to rows that satisfy mask_service
tbl_BP.loc[mask_dhl, 'ShipDate'] = pd.to_datetime(tbl_BP.loc[mask_dhl, 'ShipDate'])
tbl_BP.loc[mask_dhl, 'longest_side'] = tbl_BP.loc[mask_dhl, ['L', 'W', 'H']].max(axis=1)

# Compute constant conditions
date_conditions_1 = (tbl_BP.loc[mask_dhl, 'ShipDate'] >= nqd_dhl_date_1) & (tbl_BP.loc[mask_dhl, 'ShipDate'] <= nqd_dhl_date_2)
date_conditions_2 = (tbl_BP.loc[mask_dhl, 'ShipDate'] >= nqd_dhl_date_3) & (tbl_BP.loc[mask_dhl, 'ShipDate'] <= nqd_dhl_date_4)
actual_weight_condition = tbl_BP.loc[mask_dhl, 'ActualWeight'] >= dhl_weight_threshold
cubic_inch_condition = tbl_BP.loc[mask_dhl, 'CubicInch'] >= dhl_cubic_inch_threshold_1
girth_condition = (tbl_BP.loc[mask_dhl, 'GirthAndL'] > dhl_g_and_l_threshold_1) & (tbl_BP.loc[mask_dhl, 'GirthAndL'] <= dhl_g_and_l_threshold_2)

# Compute the 'NQD_DHLG_1' column only for rows in df that satisfy the mask_service condition
tbl_BP.loc[mask_dhl, 'NQD_DHLG_1'] = np.where(actual_weight_condition & cubic_inch_condition & date_conditions_2, 
                                                  2.5 * tbl_BP.loc[mask_dhl, 'CustomerBilledWeightLb'], 
                                                  np.where(actual_weight_condition & cubic_inch_condition & date_conditions_1, 
                                                           2 * tbl_BP.loc[mask_dhl, 'CustomerBilledWeightLb'], 0))

tbl_BP.loc[mask_dhl, 'NQD_DHLG_2'] = np.where(girth_condition & cubic_inch_condition & actual_weight_condition & date_conditions_2, 
                                                  2.5 * tbl_BP.loc[mask_dhl, 'CustomerBilledWeightLb'], 
                                                  np.where(girth_condition & cubic_inch_condition & actual_weight_condition & date_conditions_1, 
                                                           2 * tbl_BP.loc[mask_dhl, 'CustomerBilledWeightLb'], 
                                                           np.where(girth_condition & (~cubic_inch_condition | ~actual_weight_condition) & date_conditions_2, 
                                                                    2.5 * np.ceil(tbl_BP.loc[mask_dhl, 'ActualWeight']),
                                                                    np.where(girth_condition & (~cubic_inch_condition | ~actual_weight_condition) & date_conditions_1, 
                                                                             2 * np.ceil(tbl_BP.loc[mask_dhl, 'ActualWeight']),
                                                                             np.where(tbl_BP.loc[mask_dhl,'GirthAndL'] > dhl_g_and_l_threshold_2, 
                                                                                      np.nan, 0)))))

tbl_BP.loc[mask_dhl, 'NQD_DHLG_3'] = np.where((tbl_BP.loc[mask_dhl, 'longest_side'] > dhl_max_l_threshold_2) & cubic_inch_condition & actual_weight_condition & date_conditions_2, 
                                                  2.5 * tbl_BP.loc[mask_dhl, 'CustomerBilledWeightLb'], 
                                                  np.where((tbl_BP.loc[mask_dhl, 'longest_side'] > dhl_max_l_threshold_2) & cubic_inch_condition & actual_weight_condition & date_conditions_1, 
                                                           2 * tbl_BP.loc[mask_dhl, 'CustomerBilledWeightLb'],
                                                           np.where((tbl_BP.loc[mask_dhl, 'longest_side'] > dhl_max_l_threshold_2) & (~cubic_inch_condition | ~actual_weight_condition) & date_conditions_2, 
                                                                    2.5 * np.ceil(tbl_BP.loc[mask_dhl, 'ActualWeight']),
                                                                    np.where((tbl_BP.loc[mask_dhl, 'longest_side'] > dhl_max_l_threshold_2) & (~cubic_inch_condition | ~actual_weight_condition) & date_conditions_1, 
                                                                             2 * np.ceil(tbl_BP.loc[mask_dhl, 'ActualWeight']), 0))))


tbl_BP.loc[mask_dhl, 'NQD_DHLG_4'] = np.where((tbl_BP.loc[mask_dhl, 'longest_side'] > dhl_max_l_threshold_1) & (tbl_BP.loc[mask_dhl, 'longest_side'] <= dhl_max_l_threshold_3), dhl_nqd_lengthfee_1, 
                                                  np.where(tbl_BP.loc[mask_dhl, 'longest_side'] > dhl_max_l_threshold_3, 
                                                           dhl_nqd_lengthfee_2, 0))

tbl_BP.loc[mask_dhl, 'NQD_DHLG_5'] = np.where(tbl_BP.loc[mask_dhl, 'CubicInch'] > dhl_cubic_inch_threshold_2, dhl_nqd_volumefee, 0)

cols_to_consolidate = ['NQD_DHLG_1', 'NQD_DHLG_2', 'NQD_DHLG_3', 'NQD_DHLG_4', 'NQD_DHLG_5']



# Check for NaN values in 'NQD_DHLG_2'
nan_condition = (tbl_BP['NQD_DHLG_2'].isna()) & mask_dhl
# Compute the sum across the specified columns
tbl_BP.loc[mask_dhl, 'BarrettExtra'] = tbl_BP.loc[mask_dhl, cols_to_consolidate].sum(axis=1)
# If 'NQD_DHLG_2' is NaN, then set 'CustomerExtra' to NaN
tbl_BP.loc[nan_condition, 'BarrettExtra'] = np.nan

tbl_BP = tbl_BP.drop(columns=['NQD_DHLG_1', 'NQD_DHLG_2', 'NQD_DHLG_3', 'NQD_DHLG_4', 'NQD_DHLG_5', 'longest_side'])



service_codes = ["USPSPS", "USPSFC", "USPSAP"]
mask_usps = tbl_BP['ServiceCode'].isin(service_codes)

conditions = [(tbl_BP['L'] > usps_l_threshold_1) & (tbl_BP['L'] <= usps_l_threshold_2), tbl_BP['L'] > usps_l_threshold_2]
choices = [usps_nonestandard_lengthfee_1, usps_nonestandard_lengthfee_2]
tbl_BP['EXTRA_1'] = np.select(conditions, choices, default=0)

condition = tbl_BP['CubicInch'] > usps_cubic_inch_threshold
tbl_BP['EXTRA_2'] = np.where(condition, usps_nonestandard_volumefee, 0)

# Summing up the two new columns' values and storing in the "Extra" column for the desired subset of rows
tbl_BP.loc[mask_usps, 'BarrettExtra'] = tbl_BP.loc[mask_usps, 'EXTRA_1'] + tbl_BP.loc[mask_usps, 'EXTRA_2']

# If you don't need the two additional columns afterwards, you can drop them
tbl_BP = tbl_BP.drop(columns=['EXTRA_1', 'EXTRA_2'])



In [64]:
#Get CustomerExtra from BarrettExtra 
filtered_ClientCharge = ClientCharge[ClientCharge['ServiceCode'].isin(['DHLG', 'DHLG<1', 'DHLE', 'DHLE<1', 'DHLEM', 'USPSFC', 'USPSPS', 'USPSAP'])][['CompanyID', 'ServiceCode', 'Price']].drop_duplicates(subset=['CompanyID', 'ServiceCode', 'Price'])

# Define the specific ServiceCode values you're interested in
specific_servicecodes = ["DHLG", "DHLG<1", "DHLE", "DHLE<1", "DHLEM", "USPSFC", "USPSPS", "USPSAP"]


# Step 1: Create the intermediate dataframe
intermediate_df = tbl_BP.query('ServiceCode in @specific_servicecodes').merge(
    filtered_ClientCharge, 
    left_on=['CustomerID', 'ServiceCode'], 
    right_on=['CompanyID', 'ServiceCode'], 
    how='left'
)

intermediate_df['Price'] = intermediate_df['Price'].astype(float)

# Calculate 'CustomerExtra', setting it to NaN where 'Price' is NaN
intermediate_df['CustomerExtra'] = np.where(
    intermediate_df['Price'].isna(),
    np.nan,
    np.round(intermediate_df['BarrettExtra'] / (1 - intermediate_df['Price']), 2)
)


# attached the CustomerFreight back to the original PLD
tbl_BP = tbl_BP.merge(
    intermediate_df[['TrackingNumber', 'CustomerExtra', 'Price']],
    left_on=['TrackingNumber'],
    right_on=['TrackingNumber'],
    how='left'
)



# Fill NaN in the specified columns with 0 for easier calculation later
tbl_BP['CustomerExtra'] = tbl_BP['CustomerExtra'].fillna(0)



***i. Get FSC (only for UPS and DHL)***

In [65]:
# Specify the ServiceCodes for the three scenarios
specific_servicecodes_upsFsc = ["REDE", "RED", "REDS", "2DAM", "BLUE", "ORNG", "GRND", "GRES", "SRPT", "SRPT<1", "CNST", "WWX", "WWXS", "WWE"]  
specific_servicecodes_dhlFsc = ["DHLG", "DHLG<1", "DHLE", "DHLE<1", "DHLEM"]  


# Compute PublishedFsc for UPS
mask_upsFsc = tbl_BP['ServiceCode'].isin(specific_servicecodes_upsFsc)
tbl_BP.loc[mask_upsFsc, 'PublishedFsc'] = round((tbl_BP['PublishedFreight'] + tbl_BP['PublishedRes'] + tbl_BP['PublishedDas'] + tbl_BP['PublishedAhs']) * tbl_BP['FscRate'],2)

# Compute CustomerFsc for UPS
tbl_BP.loc[mask_upsFsc, 'CustomerFsc'] = round((tbl_BP['CustomerFreight'] + tbl_BP['CustomerRes'] + tbl_BP['CustomerDas'] + tbl_BP['CustomerAhs']) * tbl_BP['FscRate'],2)

# Compute CustomerFsc for DHL
mask_dhlFsc = tbl_BP['ServiceCode'].isin(specific_servicecodes_dhlFsc)
tbl_BP.loc[mask_dhlFsc, 'CustomerFsc'] = np.where((tbl_BP.loc[mask_dhlFsc, 'ActualWeight'] <= 1), (tbl_BP.loc[mask_dhlFsc, 'FscRate'] * tbl_BP.loc[mask_dhlFsc, 'ActualWeight']) / (1 - tbl_BP.loc[mask_dhlFsc, 'Price']),
         np.where((tbl_BP.loc[mask_dhlFsc, 'ActualWeight'] > 1), (tbl_BP.loc[mask_dhlFsc, 'FscRate'] * tbl_BP.loc[mask_dhlFsc, 'CustomerBilledWeightLb']) / (1 - tbl_BP.loc[mask_dhlFsc, 'Price']), 0))


# Compute BarrettFsc for UPS
tbl_BP.loc[mask_upsFsc, 'BarrettFsc'] = round((tbl_BP['BarrettFreight'] + tbl_BP['BarrettRes'] + tbl_BP['BarrettDas'] + tbl_BP['BarrettAhs']) * (tbl_BP['FscRate']*ups_fsc_disc),2)

# Compute BarrettFsc for DHL
mask_dhlFsc = tbl_BP['ServiceCode'].isin(specific_servicecodes_dhlFsc)
tbl_BP.loc[mask_dhlFsc, 'BarrettFsc'] = np.where((tbl_BP.loc[mask_dhlFsc, 'ActualWeight'] <= 1), (tbl_BP.loc[mask_dhlFsc, 'FscRate'] * tbl_BP.loc[mask_dhlFsc, 'ActualWeight']),
         np.where((tbl_BP.loc[mask_dhlFsc, 'ActualWeight'] > 1), (tbl_BP.loc[mask_dhlFsc, 'FscRate'] * tbl_BP.loc[mask_dhlFsc, 'CustomerBilledWeightLb']), 0))




***j. Get Total***

In [66]:
# Specify the ServiceCodes for the three scenarios
specific_servicecodes_UPS = ["REDE", "RED", "REDS", "2DAM", "BLUE", "ORNG", "GRND", "GRES", "SRPT", "SRPT<1", "CNST", "WWE", "WWXS", "WWX"]  

specific_servicecodes_DHL = ["DHLG", "DHLG<1", "DHLE", "DHLE<1", "DHLEM"]  

specific_servicecodes_USPS = ["USPSFC", "USPSPS", "USPSAP"]  


# Compute PublishedTotal for UPS
mask_UPS = tbl_BP['ServiceCode'].isin(specific_servicecodes_UPS)
tbl_BP.loc[mask_UPS, 'PublishedTotal'] = round(tbl_BP['PublishedFreight'] + tbl_BP['PublishedRes'] + tbl_BP['PublishedDas'] + tbl_BP['PublishedAhs'] + tbl_BP['PublishedFsc'],2)

# Compute CustomerTotal for UPS
tbl_BP.loc[mask_UPS, 'CustomerTotal'] = round(tbl_BP['CustomerFreight'] + tbl_BP['CustomerRes'] + tbl_BP['CustomerDas'] + tbl_BP['CustomerAhs'] + tbl_BP['CustomerFsc'],2)

# Compute CustomerTotal for DHL
mask_DHL = tbl_BP['ServiceCode'].isin(specific_servicecodes_DHL)
tbl_BP.loc[mask_DHL, 'CustomerTotal'] = round(tbl_BP['CustomerFreight'] + tbl_BP['CustomerExtra'] + tbl_BP['CustomerFsc'],2)

# Compute CustomerTotal for USPS
mask_USPS = tbl_BP['ServiceCode'].isin(specific_servicecodes_USPS)
tbl_BP.loc[mask_USPS, 'CustomerTotal'] = round(tbl_BP['CustomerFreight'] + tbl_BP['CustomerExtra'],2)

# Compute BarrettTotal for UPS
tbl_BP.loc[mask_UPS, 'BarrettTotal'] = round(tbl_BP['BarrettFreight'] + tbl_BP['BarrettRes'] + tbl_BP['BarrettDas'] + tbl_BP['BarrettAhs'] + tbl_BP['BarrettFsc'],2)

# Compute BarrettTotal for DHL
mask_DHL = tbl_BP['ServiceCode'].isin(specific_servicecodes_DHL)
tbl_BP.loc[mask_DHL, 'BarrettTotal'] = round(tbl_BP['BarrettFreight'] + tbl_BP['BarrettExtra'] + tbl_BP['BarrettFsc'],2)

# Compute BarrettTotal for USPS
mask_USPS = tbl_BP['ServiceCode'].isin(specific_servicecodes_USPS)
tbl_BP.loc[mask_USPS, 'BarrettTotal'] = round(tbl_BP['BarrettFreight'] + tbl_BP['BarrettExtra'],2)


# Filter rows where 'exclude_rates' column value is in the provided list
condition = ~tbl_BP['ServiceCode'].isin(["REDE", "RED", "REDS", "2DAM", "BLUE", "ORNG", "GRND", "GRES", "SRPT", "SRPT<1", "CNST", "WWE", "WWXS", "WWX"])

# If the condition is met, then set 'PublishedFreight' to NaN
tbl_BP.loc[condition, 'PublishedFreight'] = np.nan
tbl_BP.loc[condition, 'PublishedRes'] = np.nan
tbl_BP.loc[condition, 'PublishedDas'] = np.nan
tbl_BP.loc[condition, 'PublishedAhs'] = np.nan
tbl_BP.loc[condition, 'PublishedFsc'] = np.nan

# If you don't need the two additional columns afterwards, you can drop them
tbl_BP = tbl_BP.drop(columns=['Price', 'PublishedFreightTemp'])



***k. Handle Special Cases and Flag Rate***

In [67]:
ClientChargeSpecial = ClientCharge[(ClientCharge['PriceType'] == 'Special')]

In [68]:
# Johnnie-o parameters

flatJo_1 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 9019.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['RED', 'REDS'])), 'PriceMin'].iloc[0]

flatJo_2 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 9019.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['2DAM', 'BlUE'])), 'PriceMin'].iloc[0]

CAJo = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 9019.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['WWX', 'WWXS', 'WWE'])), 'Price'].iloc[0]


# HandleJohnnie-o flat rate special case
tbl_BP.loc[
    (tbl_BP['CustomerID'].isin([9019, 90190])) &
    (tbl_BP['ServiceCode'].isin(['RED', 'REDS'])) &
    (tbl_BP['CustomerBilledWeightLb'] <= 5) &
    (tbl_BP['Zone'].isin(['1','2','3','4','5','6','7','8'])),
    'CustomerFreight'
] = flatJo_1

tbl_BP.loc[
    (tbl_BP['CustomerID'].isin([9019, 90190])) &
    (tbl_BP['ServiceCode'].isin(['2DAM','BLUE'])) &
    (tbl_BP['CustomerBilledWeightLb'] <= 5) &
    (tbl_BP['Zone'].isin(['1','2','3','4','5','6','7','8'])),
    'CustomerFreight'
] = flatJo_2

tbl_BP.loc[
    (tbl_BP['CustomerID'] == 9019) &
    (tbl_BP['ServiceCode'].isin(['RED','REDS'])) &
    (tbl_BP['CustomerFreight'] == flatJo_1),
    'CustomerTotal'
] = flatJo_1


tbl_BP.loc[
    (tbl_BP['CustomerID'] == 9019) &
    (tbl_BP['ServiceCode'].isin(['2DAM','BLUE'])) &
    (tbl_BP['CustomerFreight'] == flatJo_2),
    'CustomerTotal'
] = flatJo_2


tbl_BP.loc[
    (tbl_BP['CustomerID'] == 9019) &
    (tbl_BP['ServiceCode'].isin(['RED','REDS'])) &
    (tbl_BP['CustomerFreight'] == flatJo_1),
    ['CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerFsc']
] = [0, 0, 0, 0]


tbl_BP.loc[
    (tbl_BP['CustomerID'] == 9019) &
    (tbl_BP['ServiceCode'].isin(['2DAM','BLUE'])) &
    (tbl_BP['CustomerFreight'] == flatJo_2),
    ['CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerFsc']
] = [0, 0, 0, 0]



tbl_BP.loc[
    (tbl_BP['CustomerID'].isin([9019, 90190])) &
    (tbl_BP['ServiceCode'].isin(['WWX','WWXS','WWE'])) &
    (tbl_BP['ShipToCountry'] == 'CA'),
    'Price'
] = CAJo


condition_jo = (
            (tbl_BP['CustomerID'].isin([9019, 90190])) & 
            (tbl_BP['ServiceCode'].isin(['WWX','WWXS','WWE'])) & 
            (tbl_BP['ShipToCountry'] == 'CA'))
        
tbl_BP.loc[condition_jo, 'CustomerFreight'] = tbl_BP.loc[condition_jo, 'PublishedFreight'] * (1 - CAJo)

tbl_BP.loc[condition_jo, 'CustomerFsc'] = (tbl_BP.loc[condition_jo, 'CustomerFreight']  +
                                        tbl_BP.loc[condition_jo, 'CustomerRes'] +
                                        tbl_BP.loc[condition_jo, 'CustomerDas'] + 
                                        tbl_BP.loc[condition_jo, 'CustomerAhs'] +
                                        tbl_BP.loc[condition_jo, 'CustomerExtra']) * (tbl_BP.loc[condition_jo, 'FscRate']) 

tbl_BP.loc[condition_jo, 'CustomerTotal'] = (tbl_BP.loc[condition_jo, 'CustomerFreight']  +
                                          tbl_BP.loc[condition_jo, 'CustomerRes'] +
                                          tbl_BP.loc[condition_jo, 'CustomerDas'] + 
                                          tbl_BP.loc[condition_jo, 'CustomerAhs'] +
                                          tbl_BP.loc[condition_jo, 'CustomerExtra'] +
                                          tbl_BP.loc[condition_jo, 'CustomerFsc'])


In [69]:
# Ardent parameters
flatArdent_1 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 1127.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['ORNG'])), 'PriceMin'].iloc[0]

flatArdent_2 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 1127.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['ORNG'])), 'PriceMin'].iloc[1]

flatArdent_3 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 1127.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['GRND', 'GRES'])), 'PriceMin'].iloc[0]

flatArdent_4 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 1127.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['GRND', 'GRES'])), 'PriceMin'].iloc[1]

minArdent = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 1127.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['ORNG'])), 'PriceMin'].iloc[2]

discArdent = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 1127.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['ORNG'])), 'Price'].iloc[2]


# Ardent calculation
conditions_ard_1 = [
    (tbl_BP['CustomerID'] == 1127) & (tbl_BP['ServiceCode'].isin(['ORNG'])) & (tbl_BP['CustomerBilledWeightLb'] >= 1) & (tbl_BP['CustomerBilledWeightLb'] <= 7),
    (tbl_BP['CustomerID'] == 1127) & (tbl_BP['ServiceCode'].isin(['ORNG'])) & (tbl_BP['CustomerBilledWeightLb'] >= 8) & (tbl_BP['CustomerBilledWeightLb'] <= 10)
]

conditions_ard_2 = [
    (tbl_BP['CustomerID'] == 1127) & (tbl_BP['ServiceCode'].isin(['GRND', 'GRES'])) & (tbl_BP['CustomerBilledWeightLb'] >= 1) & (tbl_BP['CustomerBilledWeightLb'] <= 7),
    (tbl_BP['CustomerID'] == 1127) & (tbl_BP['ServiceCode'].isin(['GRND', 'GRES'])) & (tbl_BP['CustomerBilledWeightLb'] >= 8) & (tbl_BP['CustomerBilledWeightLb'] <= 10)
]

# Define the corresponding actions for each condition
choices_1 = [
    flatArdent_1,  
    flatArdent_2  
    
]

choices_2 = [
   flatArdent_3,  
   flatArdent_4 
    
]

choices_3 = [
    0,  
    0  
]


# Apply the conditions and choices to the dataframe
tbl_BP['CustomerFreight'] = np.select(conditions_ard_1, choices_1, default=tbl_BP['CustomerFreight'])
tbl_BP['CustomerFreight'] = np.select(conditions_ard_2, choices_2, default=tbl_BP['CustomerFreight'])
tbl_BP['CustomerRes'] = np.select(conditions_ard_1, choices_3, default=tbl_BP['CustomerRes'])
tbl_BP['CustomerDas'] = np.select(conditions_ard_1, choices_3, default=tbl_BP['CustomerDas'])
tbl_BP['CustomerFsc'] = np.select(conditions_ard_1, choices_3, default=tbl_BP['CustomerFsc'])


def calculate_customer_total(row):
    if row['CustomerID'] == 1127 and row['ServiceCode'] in ['ORNG', 'GRND', 'GRES']:
        if 1 <= row['CustomerBilledWeightLb'] <= 10:
            return row['CustomerFreight'] + row['CustomerAhs']
    return row['CustomerTotal']

tbl_BP['CustomerTotal'] = tbl_BP.apply(calculate_customer_total, axis=1)


def custom_function(row):
    if row['CustomerID'] == 1127 and row['ServiceCode'] == 'ORNG' and row['CustomerBilledWeightLb'] >= 11:
        return round(max(row['PublishedFreight'] * (1 - discArdent), minArdent), 2)
    else:
        return row['CustomerFreight']

tbl_BP['CustomerFreight'] = tbl_BP.apply(custom_function, axis=1)

# Recalculate CustomerFsc and CustomerTotal within the same loop
for index, row in tbl_BP.iterrows():
    if row['CustomerID'] == 1127 and row['ServiceCode'] == 'ORNG' and row['CustomerBilledWeightLb'] >= 11:
        # Calculate CustomerFsc based on the current row's values
        customer_fsc = (row['CustomerFreight'] + row['CustomerRes'] + row['CustomerDas'] + 
                        row['CustomerAhs'] + row['CustomerExtra']) * row['FscRate']
        tbl_BP.at[index, 'CustomerFsc'] = round(customer_fsc,2)

        # Calculate CustomerTotal based on the updated CustomerFsc
        customer_total = (row['CustomerFreight'] + row['CustomerRes'] + row['CustomerDas'] + 
                          row['CustomerAhs'] + row['CustomerExtra'] + customer_fsc)
        tbl_BP.at[index, 'CustomerTotal'] = round(customer_total,2)
      

In [70]:
# Erms sport parameters
flatErms_1 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 6483.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['SRPT<1'])), 'PriceMin'].iloc[0]

flatErms_2 = ClientChargeSpecial.loc[(ClientChargeSpecial['CompanyID'] == 6483.0) &
                 (ClientChargeSpecial['ServiceCode'].isin(['SRPT'])), 'PriceMin'].iloc[0]


# Erms sport calculation
condition_es_1 = (
    (tbl_BP['CustomerID'] == 6483) &
    (tbl_BP['ServiceCode'] == 'SRPT<1') &
    (tbl_BP['CustomerBilledWeightOz'] <= 16)
)
        
condition_es_2 = (
    (tbl_BP['CustomerID'] == 6483) &
    (tbl_BP['ServiceCode'] == 'SRPT') &
    ((tbl_BP['CustomerBilledWeightLb'] >= 1) & (tbl_BP['CustomerBilledWeightLb'] <= 2))
)
        
        
tbl_BP.loc[condition_es_1, 'CustomerFreight'] = round(flatErms_1,2)
tbl_BP.loc[condition_es_2, 'CustomerFreight'] = round(flatErms_2,2)
tbl_BP.loc[condition_es_1, 'CustomerTotal'] = round(flatErms_1,2)
tbl_BP.loc[condition_es_2, 'CustomerTotal'] = round(flatErms_2,2)


tbl_BP.loc[condition_es_1,
    ['CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerFsc', 'CustomerExtra']
] = [0, 0, 0, 0, 0]

tbl_BP.loc[condition_es_2,
    ['CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerFsc', 'CustomerExtra']
] = [0, 0, 0, 0, 0]


In [71]:
# International
#Res
condition_intl_ground_res = (
    (tbl_BP['ServiceCode'] == "CNST") &
    (tbl_BP['ResidentialFlag'] == True)
)

condition_intl_air_res = (
    (tbl_BP['ServiceCode'].isin(["WWX", "WWXS", "WWE"])) &
    (tbl_BP['ResidentialFlag'] == True)
)

tbl_BP.loc[condition_intl_ground_res,
    ['PublishedRes', 'CustomerRes', 'BarrettRes']
] = [intl_ground_res, intl_ground_res, intl_ground_res] 

tbl_BP.loc[condition_intl_air_res,
    ['PublishedRes', 'CustomerRes', 'BarrettRes']
] = [intl_air_res, intl_air_res, intl_air_res]


#Ahs
# Define your conditions
condition_intl_ahs_1 = (tbl_BP['ServiceCode'].isin(["CNST","WWX", "WWXS", "WWE"])) & (tbl_BP['GirthAndL'] > 105)
condition_intl_ahs_2 = (tbl_BP['ServiceCode'].isin(["CNST","WWX", "WWXS", "WWE"])) & (tbl_BP['L'] > 48)
condition_intl_ahs_3 = (tbl_BP['ServiceCode'].isin(["CNST","WWX", "WWXS", "WWE"])) & (tbl_BP['W'] > 30)
condition_intl_ahs_4 = (tbl_BP['ServiceCode'].isin(["CNST","WWX", "WWXS", "WWE"])) & (tbl_BP['ActualWeight'] > intl_ahs_weight_limit)

# Count the number of conditions met for each row
conditions_met = condition_intl_ahs_1.astype(int) + condition_intl_ahs_2.astype(int) + condition_intl_ahs_3.astype(int) + condition_intl_ahs_4.astype(int)

# Calculate the additional amount to be added (charge for each condition met)
additional_amount = conditions_met * intl_ahs

# Add the additional amount to the price columns
tbl_BP['PublishedAhs'] += additional_amount
tbl_BP['CustomerAhs'] += additional_amount
tbl_BP['BarrettAhs'] += additional_amount
  
#Fsc
tbl_BP.loc[tbl_BP['ServiceCode'].isin(["CNST", "WWX", "WWXS", "WWE"]), 'PublishedFsc'] = round((tbl_BP['PublishedFreight'] + tbl_BP['PublishedRes'] + tbl_BP['PublishedDas'] + tbl_BP['PublishedAhs']) * tbl_BP['FscRate'],2)
tbl_BP.loc[tbl_BP['ServiceCode'].isin(["CNST", "WWX", "WWXS", "WWE"]), 'BarrettFsc'] = round((tbl_BP['BarrettFreight'] + tbl_BP['BarrettRes'] + tbl_BP['BarrettDas'] + tbl_BP['BarrettAhs']) * (tbl_BP['FscRate']*ups_fsc_disc),2)
tbl_BP.loc[tbl_BP['ServiceCode'].isin(["CNST", "WWX", "WWXS", "WWE"]), 'CustomerFsc'] = round((tbl_BP['CustomerFreight'] + tbl_BP['CustomerRes'] + tbl_BP['CustomerDas'] + tbl_BP['CustomerAhs']) * tbl_BP['FscRate'],2)


#Total PublishedTotal for UPS
tbl_BP.loc[tbl_BP['ServiceCode'].isin(["CNST", "WWX", "WWXS", "WWE"]), 'PublishedTotal'] = round(tbl_BP['PublishedFreight'] + tbl_BP['PublishedRes'] + tbl_BP['PublishedDas'] + tbl_BP['PublishedAhs'] + tbl_BP['PublishedFsc'],2)

# Compute CustomerTotal for UPS
tbl_BP.loc[tbl_BP['ServiceCode'].isin(["CNST", "WWX", "WWXS", "WWE"]), 'CustomerTotal'] = round(tbl_BP['CustomerFreight'] + tbl_BP['CustomerRes'] + tbl_BP['CustomerDas'] + tbl_BP['CustomerAhs'] + tbl_BP['CustomerFsc'],2)

# Compute BarrettTotal for UPS
tbl_BP.loc[tbl_BP['ServiceCode'].isin(["CNST", "WWX", "WWXS", "WWE"]), 'BarrettTotal'] = round(tbl_BP['BarrettFreight'] + tbl_BP['BarrettRes'] + tbl_BP['BarrettDas'] + tbl_BP['BarrettAhs'] + tbl_BP['BarrettFsc'],2)




***l. Calculate the savings and Savings % and Format***

In [72]:
# Columns you want to fill
cols_to_update_1 = ['CustomerFreight', 'CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerExtra', 'CustomerFsc', 'BarrettMargin', 'CustomerSaving', 'CustomerSavingPct']
tbl_BP.loc[tbl_BP['CustomerTotal'].isnull(), cols_to_update_1] = np.nan

# Columns you want to fill
cols_to_update_2 = ['PublishedFreight', 'PublishedRes', 'PublishedDas', 'PublishedAhs', 'PublishedFsc', 'CustomerSaving', 'CustomerSavingPct']
tbl_BP.loc[tbl_BP['PublishedTotal'].isnull(), cols_to_update_2] = np.nan

# Columns you want to fill
cols_to_update_3 = ['BarrettFreight', 'BarrettRes', 'BarrettDas', 'BarrettAhs', 'BarrettExtra', 'BarrettFsc', 'BarrettMargin']
tbl_BP.loc[tbl_BP['BarrettTotal'].isnull(), cols_to_update_3] = np.nan

# Columns you want to fill
cols_to_update_4 = ['CustomerFreight', 'CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerExtra', 'CustomerFsc', 'BarrettMargin', 'CustomerSaving', 'CustomerSavingPct']
tbl_BP.loc[tbl_BP['CustomerTotal'].notnull(), cols_to_update_4] = tbl_BP.loc[tbl_BP['CustomerTotal'].notnull(), cols_to_update_4].fillna(0)

# Columns you want to fill
cols_to_update_5 = ['PublishedFreight', 'PublishedRes', 'PublishedDas', 'PublishedAhs', 'PublishedFsc', 'CustomerSaving', 'CustomerSavingPct']
tbl_BP.loc[tbl_BP['PublishedTotal'].notnull(), cols_to_update_5] = tbl_BP.loc[tbl_BP['PublishedTotal'].notnull(), cols_to_update_5].fillna(0)

# Columns you want to fill
cols_to_update_6 = ['BarrettFreight', 'BarrettRes', 'BarrettDas', 'BarrettAhs', 'BarrettExtra', 'BarrettFsc', 'BarrettMargin']
tbl_BP.loc[tbl_BP['BarrettTotal'].notnull(), cols_to_update_6] = tbl_BP.loc[tbl_BP['BarrettTotal'].notnull(), cols_to_update_6].fillna(0)


# Calculate for Customer Resi, Das, and Ahs rates
tbl_BP['CustomerSaving'] = round(tbl_BP['PublishedTotal'] - tbl_BP['CustomerTotal'],2)
tbl_BP['CustomerSavingPct'] = round((tbl_BP['PublishedTotal'] - tbl_BP['CustomerTotal'])/tbl_BP['PublishedTotal'],2)
tbl_BP['BarrettMargin'] = round(1 - tbl_BP['BarrettTotal']/tbl_BP['CustomerTotal'],2)

tbl_BP = tbl_BP.drop(columns=['Price'])

# Columns to format
cols_to_format_1 = ['CustomerFreight', 'CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerExtra', 'CustomerFsc', 'CustomerTotal', 
                    'PublishedFreight', 'PublishedRes', 'PublishedDas', 'PublishedAhs', 'PublishedFsc', 'PublishedTotal', 
                    'BarrettFreight', 'BarrettRes', 'BarrettDas', 'BarrettAhs', 'BarrettExtra', 'BarrettFsc', 'BarrettTotal',
                    'CustomerSaving']

# Columns to format
cols_to_format_2 = ['FscRate', 'CustomerSavingPct', 'BarrettMargin']

# Apply currency format
tbl_BP[cols_to_format_1] = tbl_BP[cols_to_format_1].applymap(lambda x: '${:,.2f}'.format(x) if pd.notna(x) else x)

# Apply percentage format
tbl_BP[cols_to_format_2] = tbl_BP[cols_to_format_2].applymap(lambda x: '{:.2%}'.format(x) if pd.notna(x) else x)


# Now update the RateFlag based on the conditions. There is no possibility that SRPT<1 has lb rather than oz value. 
tbl_BP.loc[
    (((tbl_BP['ServiceCode'].isin(['REDE', 'RED', 'REDS', '2DAM', 'BLUE', 'ORNG', 'GRND', 'GRES', 'CNST', 'WWXS', 'WWX', 'WWE'])) & tbl_BP['CustomerBilledWeightLb'] > 150) |
     ((tbl_BP['ServiceCode'] == 'SRPT') & (tbl_BP['CustomerBilledWeightLb'] > 15)) |
     ((tbl_BP['ServiceCode'].isin(['DHLG', 'DHLE'])) & (tbl_BP['CustomerBilledWeightLb'] > 25)) |
     ((tbl_BP['ServiceCode'].isin(['MIPH', 'USPSAP'])) & (tbl_BP['CustomerBilledWeightLb'] > 70)) |
     ((tbl_BP['ServiceCode'].isin(['SRPT<1', 'DHLG<1', 'DHLE<1', 'MIPLE', 'USPSFC'])) & (tbl_BP['CustomerBilledWeightOz'] > 16)) |
     ((tbl_BP['ServiceCode'] == 'DHLEM') & (tbl_BP['CustomerBilledWeightOz'] > 400))),
    'RateFlag'
] = 0
tbl_BP.loc[(tbl_BP.Zone == '35'), 'RateFlag'] = 0



lessThan1_services = ['SRPT<1', 'DHLG<1', 'DHLE<1', 'MIPLE', 'USPSFC']  # Add your valid services here

# Update 'RateFlag' to 0 for rows where 'Service' is in the LessThan1_services
tbl_BP.loc[(tbl_BP['ServiceCode'].isin(lessThan1_services)) & (tbl_BP['ActualWeight'] >= 1), 'RateFlag'] = 0


columns_are_null = ['CustomerTotal']

# Create a condition that checks if any of the specified columns are null
condition = tbl_BP[columns_are_null].isnull().any(axis=1)

# Set 'RateFlag' to 0 where the condition is True
tbl_BP.loc[condition, 'RateFlag'] = 0


# As long as there is "Total Rate", all other corresponding null rate columns will be filled as 0 for easy calculation. 
# As long as there is no total rate, all other corresponding rate columns will be null. 
columns_to_null = ['CustomerFreight', 'CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerExtra', 'CustomerFsc', 'CustomerTotal', 
                   'PublishedFreight', 'PublishedRes', 'PublishedDas', 'PublishedAhs', 'PublishedFsc', 'PublishedTotal',
                   'BarrettFreight', 'BarrettRes', 'BarrettDas', 'BarrettAhs', 'BarrettExtra', 'BarrettFsc', 'BarrettTotal', 
                   'CustomerSaving', 'CustomerSavingPct', 'BarrettMargin']

# Set the specified columns to null where the trigger column has the specific value
tbl_BP.loc[tbl_BP['RateFlag'] == 0, columns_to_null] = np.nan


# The output columns must be below order. 
tbl_BP = tbl_BP.reindex(columns=['CustomerID', 'CustomerName', 'Facility', 'BarrettOrderNumber', 'Reference', 'PoNumber', 
       'TrackingNumber', 'ShipDate', 'Service', 'Shipper', 'JoChannel', 'ShipToName', 'ShipToContact', 'ShipToCity', 'ShipToState',
       'ZipCode', 'ShipToCountry', 'Zone', 'Quantity', 'Dimensions', 'ActualWeight', 'ResidentialFlag', 'FromZip', 'RateFlag', 
       'L', 'W', 'H', 'GirthAndL', 'CubicInch', 'ServiceCode', 'DASCategory', 'FscRate', 'CustomerBilledWeightLb', 
       'CustomerBilledWeightOz', 'CustomerFreight', 'CustomerRes', 'CustomerDas', 'CustomerAhs', 'CustomerExtra', 
       'CustomerFsc', 'CustomerTotal', 'PublishedBilledWeightLb', 'PublishedBilledWeightOz', 'PublishedFreight', 
       'PublishedRes', 'PublishedDas', 'PublishedAhs', 'PublishedFsc', 'PublishedTotal', 'BarrettFreight', 'BarrettRes', 
       'BarrettDas', 'BarrettAhs', 'BarrettExtra', 'BarrettFsc', 'BarrettTotal', 'CustomerSaving', 'CustomerSavingPct', 
       'BarrettMargin'])


In [73]:
tbl_BP.to_csv('RateCurrent_Automation_2023.csv')

In [74]:
tbl_BP.shape

(44621, 59)

# The End